In [ ]:
# Imports and Connection
import os
import sys
import json

import micropip
await micropip.install(["hypha-rpc", "httpx"])
await micropip.install('seaborn')
import asyncio
from hypha_rpc import connect_to_server
import httpx
import time
from IPython.display import display, HTML
from IPython.display import Image as IPythonImage
from typing import Union, Optional, Tuple, Sequence, List, Dict, Any
from io import BytesIO
import io
import numpy as np
import base64
from skimage.measure import regionprops
from skimage import exposure
from PIL import Image as PILImage
import math

workspace_token = "YOUR_WORKSPACE_TOKEN_HERE"
# Connect to Hypha server
reef_server = await connect_to_server({
    "server_url": "https://hypha.aicell.io",
    "token": workspace_token,
    "workspace": "agent-lens",
    "ping_interval": None
})

# Get services
segmentation_service = await reef_server.get_service("agent-lens/cell-segmenter")
agent_lens_service = await reef_server.get_service("agent-lens/agent-lens-tools")

# Microscope configuration
microscope_id = "agent-lens/squid-control-simulation-mcp"
microscope = await reef_server.get_service(microscope_id)


# Reset ChromaDB vector database for fresh start
application_id = "hypha-agents-notebook"
reset_result = await agent_lens_service.reset_application(application_id)

# Cell 4: Core Helper Functions
# Channel canonical names
fixed_channel_order = [
    'BF_LED_matrix_full',
    'Fluorescence_405_nm_Ex',
    'Fluorescence_488_nm_Ex',
    'Fluorescence_638_nm_Ex',
    'Fluorescence_561_nm_Ex',
    'Fluorescence_730_nm_Ex'
]

color_map = {
    "0": (1.0, 1.0, 1.0),  # BF: gray
    "1": (0.0, 0.0, 1.0),  # 405nm: blue
    "2": (0.0, 1.0, 0.0),  # 488nm: green
    "3": (1.0, 0.0, 0.0),  # 638nm: red
    "4": (1.0, 1.0, 0.0),  # 561nm: yellow(simulated)
}

async def snap_image(channel_config: List[Dict[str, Any]]) -> List[Optional[np.ndarray]]:
    """Snap multi-channel images and return list of channels. Missing channels are None."""
    channel_to_idx = {ch: idx for idx, ch in enumerate(fixed_channel_order)}
    channels: List[Optional[np.ndarray]] = [None] * len(fixed_channel_order)
    
    for config in channel_config:
        channel_name = config["channel"]
        channel_idx = channel_to_idx[channel_name]
        exposure_time = config["exposure_time"]
        intensity = config["intensity"]
        
        image_np = await microscope.snap(
            channel=channel_name,
            exposure_time=exposure_time,
            intensity=intensity,
            return_array=True
        )
        channels[channel_idx] = image_np
    
    return channels


def overlay(image_channels: List[Optional[np.ndarray]], color_map) -> np.ndarray:
    """Create RGB composite from sparse channel list using additive color blending."""
    first = next((ch for ch in image_channels if ch is not None), None)
    if first is None:
        return np.zeros((1, 1, 3), dtype=np.uint8)
    
    H, W = first.shape[:2]
    
    rgb_composite = np.zeros((H, W, 3), dtype=np.float64)
    
    for channel_idx_str, (r, g, b) in color_map.items():
        channel_idx = int(channel_idx_str)  # Convert string key to integer
        ch = image_channels[channel_idx] if channel_idx < len(image_channels) else None
        if ch is None:
            continue
        channel_data = ch.astype(np.float64)
        max_val = channel_data.max()
        if max_val > 0:
            channel_data = channel_data / max_val
        
        rgb_composite[:, :, 0] += channel_data * r
        rgb_composite[:, :, 1] += channel_data * g
        rgb_composite[:, :, 2] += channel_data * b
    
    if rgb_composite.max() > 0:
        rgb_composite = (rgb_composite / rgb_composite.max() * 255).astype(np.uint8)
    else:
        rgb_composite = rgb_composite.astype(np.uint8)
    
    return rgb_composite


def percentile_normalize(
    image_data: List[Optional[np.ndarray]],
    lower_percentile: float = 1.0,
    upper_percentile: float = 99.0,
    output_dtype: type = np.uint8,
) -> List[Optional[np.ndarray]]:
    """Apply percentile normalization to all non-empty channels. Returns same list shape."""
    normalized: List[Optional[np.ndarray]] = []
    
    if output_dtype == np.uint8:
        output_min, output_max = 0, 255
    elif output_dtype == np.uint16:
        output_min, output_max = 0, 65535
    else:
        output_min, output_max = 0.0, 1.0
    
    for ch in image_data:
        if ch is None:
            normalized.append(None)
            continue
        
        channel = ch
        if channel.max() > 0:
            p_low = np.percentile(channel, lower_percentile)
            p_high = np.percentile(channel, upper_percentile)
            clipped = np.clip(channel, p_low, p_high)
            if p_high > p_low:
                norm = (clipped - p_low) / (p_high - p_low) * (output_max - output_min) + output_min
            else:
                norm = np.full_like(channel, output_min, dtype=np.float64)
        else:
            norm = np.zeros_like(channel, dtype=np.float64)
        
        normalized.append(norm.astype(output_dtype))
    
    return normalized

async def segment_image(image_data, scale: int = 8) -> np.ndarray:
    """
    Segment cells from image: BF (channel 0) if present; otherwise use overlay(image_channels) composite.
    Accepts:
      - image_data as np.ndarray (H,W,C) or (H,W)
      - or list/tuple of channels (can include None)
    """
    # ---- convert to channel list (preserve indices incl. None) ----
    if isinstance(image_data, (list, tuple)):
        chans = list(image_data)
    else:
        arr = np.asarray(image_data)
        if arr.ndim == 2:
            chans = [arr]
        else:
            chans = [arr[:, :, i] for i in range(arr.shape[2])]

    # ---- pick segmentation input (RGB, do NOT convert to grayscale) ----
    bf = chans[0] if len(chans) > 0 else None
    if bf is not None and np.nanstd(bf) > 1e-6:
        # Use BF image (single channel): promote to 3-channel grayscale RGB if needed
        if bf.dtype != np.uint8:
            g = bf.astype(np.float32)
            g = (g - np.nanmin(g)) / (np.nanmax(g) - np.nanmin(g) + 1e-12) * 255.0
            gray_u8 = np.clip(g, 0, 255).astype(np.uint8)
        else:
            gray_u8 = bf
        input_rgb = np.stack([gray_u8] * 3, axis=-1)
    else:
        # Use overlay (already RGB)
        input_rgb = overlay(chans)  # (H,W,3) uint8

    H, W = input_rgb.shape[:2]

    # ---- downscale ----
    if scale and scale > 1:
        pil_img = PILImage.fromarray(input_rgb, "RGB").resize((max(1, W // scale), max(1, H // scale)), PILImage.BILINEAR)
    else:
        pil_img = PILImage.fromarray(input_rgb, "RGB")

    # ---- encode + segment ----
    buf = BytesIO()
    pil_img.save(buf, format="PNG")
    b64 = base64.b64encode(buf.getvalue()).decode("utf-8")

    res = await segmentation_service.segment_all(b64)
    mask_small = res["mask"] if isinstance(res, dict) else res

    # ---- upscale mask ----
    if scale and scale > 1:
        mask = np.array(
            PILImage.fromarray(np.array(mask_small, np.uint16)).resize((W, H), PILImage.NEAREST)
        )
    else:
        mask = np.array(mask_small)

    return mask

async def wait_for_snap_segment_extract(agent_lens_service, poll_interval=30):
    """
    Helper function to poll snap_segment_extract status until completion.
    Returns:
        List of cell records, or empty list if idle/error
    """
    
    print('Waiting for all snap+segment+extract jobs to finish...')
    start_time = time.time()
    
    while True:
        await asyncio.sleep(3)  # Small delay before first check
        status = await agent_lens_service.poll_snap_segment_extract_status()
        
        if status['status'] == 'idle':
            print("No work in progress")
            return []
            
        elif status['status'] == 'running':
            queue_info = status['queue_sizes']
            workers_busy = status.get('workers_busy', {})
            results_so_far = status['results_count']
            elapsed = time.time() - start_time
            
            # Show worker status
            worker_status = []
            if workers_busy.get('snap_worker', False):
                worker_status.append("snap:BUSY")
            if workers_busy.get('segment_build_workers', False):
                worker_status.append("seg/build:BUSY")
            worker_str = ", ".join(worker_status) if worker_status else "all idle"
            
            print(f"[{elapsed:.1f}s] Processing... "
                  f"queues[snap:{queue_info['snap_queue']}, "
                  f"seg:{queue_info['segment_queue']}, "
                  f"build:{queue_info['build_queue']}], "
                  f"workers[{worker_str}], "
                  f"results:{results_so_far}")
            await asyncio.sleep(poll_interval)
            
        elif status['status'] == 'succeed':
            elapsed = time.time() - start_time
            print(f"✓ Complete in {elapsed:.1f}s! Got {len(status['result'])} cell records")
            return status['result']
            
        elif status['status'] == 'error':
            print(f"✗ Error occurred: {status.get('error', 'Unknown error')}")
            return []

def np_to_base64_png(arr: np.ndarray, normalize: bool = True) -> str:
    """Convert numpy array to base64 PNG string for HTML display."""
    if normalize and arr.max() > 0:
        arr = (arr / arr.max() * 255).astype(np.uint8)
    elif arr.dtype != np.uint8:
        arr = arr.astype(np.uint8)
    pil_img = PILImage.fromarray(arr)
    buf = io.BytesIO()
    pil_img.save(buf, format="PNG")
    return base64.b64encode(buf.getvalue()).decode("utf-8")


def make_stage_offsets(
    grid_size: int,
    step_mm: float = 1.0
) -> List[Tuple[float, float]]:
    """Generate (dx, dy) offsets for grid scan, sorted by distance from center."""
    origin = (grid_size - 1) / 2.0
    positions = [
        ((i - origin) * step_mm, (j - origin) * step_mm)
        for i in range(grid_size)
        for j in range(grid_size)
    ]
    positions.sort(key=lambda pos: (pos[0]**2 + pos[1]**2))
    return positions

# Filter Conversion Utilities
def convert_filters_to_chromadb_where(
    relative_config: Optional[Dict[str, float]] = None,
    range_config: Optional[Dict[str, Dict[str, float]]] = None,
    query_cell_records: Optional[List[Dict[str, Any]]] = None
) -> Optional[Dict[str, Any]]:
    """
    Convert old-style filter configs to ChromaDB where clause.
    
    Args:
        relative_config: Relative tolerances (e.g., {"size_tolerance": 0.4, "circularity_tol": 0.15})
        range_config: Absolute ranges (e.g., {"area": {"min": 200, "max": 3000}})
        query_cell_records: Query cells for computing relative filter means
    
    Returns:
        ChromaDB where clause dict, or None if no filters
    """
    conditions = []
    
    # Convert range_config (absolute filters)
    if range_config:
        for field, rule in range_config.items():
            field_conditions = []
            if "min" in rule and rule["min"] is not None:
                field_conditions.append({field: {"$gte": float(rule["min"])}})
            if "max" in rule and rule["max"] is not None:
                field_conditions.append({field: {"$lte": float(rule["max"])}})
            
            if len(field_conditions) == 1:
                conditions.append(field_conditions[0])
            elif len(field_conditions) > 1:
                conditions.append({"$and": field_conditions})
    
    # Convert relative_config (relative to query mean)
    if relative_config and query_cell_records:
        import numpy as np
        
        # Map config keys to field names and tolerance types
        filter_mappings = {
            "size_tolerance": ("area", True),  # relative
            "brightness_tol": ("brightness", False),  # absolute
            "circularity_tol": ("circularity", False),
            "aspect_ratio_tol": ("aspect_ratio", False),
            "eccentricity_tol": ("eccentricity", False),
            "solidity_tol": ("solidity", False),
        }
        
        for tol_key, (field_name, is_relative) in filter_mappings.items():
            tolerance = relative_config.get(tol_key)
            if tolerance is None:
                continue
            
            # Compute query mean
            vals = []
            for q in query_cell_records:
                v = q.get(field_name)
                if v is not None:
                    vals.append(float(v))
            
            if not vals:
                continue
            
            query_mean = float(np.mean(vals))
            
            if is_relative:
                # Relative tolerance: mean * (1 ± tolerance)
                min_val = query_mean * (1 - tolerance)
                max_val = query_mean * (1 + tolerance)
            else:
                # Absolute tolerance: mean ± tolerance
                min_val = query_mean - tolerance
                max_val = query_mean + tolerance
            
            conditions.append({
                "$and": [
                    {field_name: {"$gte": min_val}},
                    {field_name: {"$lte": max_val}}
                ]
            })
    
    # Combine all conditions
    if len(conditions) == 0:
        return None
    elif len(conditions) == 1:
        return conditions[0]
    else:
        return {"$and": conditions}


async def similarity_search_with_filters(
    query_cell_records: List[Dict[str, Any]],
    relative_config: Optional[Dict[str, float]] = None,
    range_config: Optional[Dict[str, Dict[str, float]]] = None,
    similarity_config: Optional[Dict[str, Any]] = None,
    application_id: str = "hypha-agents-notebook",
    n_results: int = 100
) -> List[Dict[str, Any]]:
    """
    Similarity search with backward-compatible filter format.
        
    Args:
        query_cell_records: Query cells to search for
        relative_config: Relative tolerances (e.g., {"size_tolerance": 0.4})
        range_config: Absolute ranges (e.g., {"area": {"min": 200, "max": 3000}})
        similarity_config: Similarity thresholds (e.g., {"final_score_threshold": 0.7})
        application_id: Vector Database application ID
        n_results: Maximum number of results
    
    Returns:
        List of similar cells with metadata and images
    """
    # Extract query UUIDs
    query_uuids = [cell["uuid"] for cell in query_cell_records if "uuid" in cell]
    
    if not query_uuids:
        print("Warning: No UUIDs found in query cells")
        return []
    
    # Convert filters to Vector Database where clause
    where_clause = convert_filters_to_chromadb_where(
        relative_config=relative_config,
        range_config=range_config,
        query_cell_records=query_cell_records
    )
    
    # Extract similarity threshold
    similarity_threshold = None
    if similarity_config:
        similarity_threshold = similarity_config.get("final_score_threshold")
    
    # Perform server-side similarity search
    similar_cells = await agent_lens_service.similarity_search_cells(
        query_cell_uuids=query_uuids,
        application_id=application_id,
        n_results=n_results,
        metadata_filters=where_clause,
        similarity_threshold=similarity_threshold
    )
    
    return similar_cells


#  Visualization Functions
def show_similarity_results(
    query_cell_records: List[Dict[str, Any]],
    similar_cells: List[Dict[str, Any]],
    max_examples: int = 20
):
    """Display similar cells from vector database search results. The first of similar_cells is the query cell."""
    
    metadata_fields = [
        ("area", "Area", "{:.1f}"),
        ("circularity", "Circ.", "{:.3f}"),
        ("similarity_score", "Sim.", "{:.3f}"),
        ("distance", "Dist.", "{:.3f}"),
    ]
    
    def cell_metadata_html(cell):
        items = []
        for k, label, fmt in metadata_fields:
            val = cell.get(k, None)
            if val is not None:
                try:
                    items.append(f"<span style='white-space:nowrap;' title='{k}'><b>{label}:</b> {fmt.format(val)}</span>")
                except:
                    pass
        
        for key in sorted(cell.keys()):
            if key.startswith('top10_mean_intensity_'):
                val = cell.get(key)
                if val is not None:
                    try:
                        channel_name = key.replace('top10_mean_intensity_', '').replace('_', ' ')
                        label = f"Top10% {channel_name}"
                        items.append(f"<span style='white-space:nowrap;' title='{key}'><b>{label}:</b> {val:.1f}</span>")
                    except:
                        pass
        
        if not items:
            return ""
        return "<div style='margin-top:4px;line-height:1.2;color:#666;font-size:9px;'>" + "<br>".join(items) + "</div>"
    
    def cell_card(cell, is_query=False):
        # Handle different image formats
        img_b64 = cell.get("image")
        if not img_b64:
            img_b64 = cell.get("image_b64")
        
        # If image is not base64 string, it might be empty or None
        if not img_b64 or not isinstance(img_b64, str):
            img_b64 = ""  # Will show broken image icon
        
        uuid = cell.get("uuid", "?")
        well = cell.get("well_id", "?")
        
        border_style = "2px solid #007bff" if is_query else "1px solid #ddd"
        metadata_html = cell_metadata_html(cell)
        
        # Show placeholder if no image
        if img_b64:
            img_html = f'<img src="data:image/png;base64,{img_b64}" style="width:120px;height:120px;object-fit:contain;"/>'
        else:
            img_html = '<div style="width:120px;height:120px;background:#f0f0f0;display:flex;align-items:center;justify-content:center;color:#999;font-size:10px;">No Image</div>'
        
        # Safe UUID display
        uuid_display = uuid[:8] if isinstance(uuid, str) and len(uuid) >= 8 else str(uuid)
        
        return f'''
        <div style="display:inline-block;margin:5px;padding:8px;border:{border_style};border-radius:4px;
                    width:160px;vertical-align:top;font-size:10px;box-sizing:border-box;">
            {img_html}
            <div style="margin-top:4px;">
                Well: {well}<br/>
                UUID: {uuid_display}...
            </div>
            {metadata_html}
        </div>'''
    
    # Assume first entry of similar_cells is the query cell, rest are actual similar cells
    if not similar_cells:
        html = "<div style='color:#999;'>No results found.</div>"
    else:
        query_cell = similar_cells[0]
        similar_sorted = sorted(similar_cells[1:], key=lambda x: -x.get("similarity_score", 0))[:max_examples]
        html = f'''
        <div style="font-family:Arial,sans-serif;">
            <h3>Query Cell</h3>
            <div>{cell_card(query_cell, is_query=True)}</div>
            <h3>Similar Cells ({len(similar_sorted)} shown, {max(0, len(similar_cells)-1)} total)</h3>
            <div>{''.join(cell_card(c) for c in similar_sorted) or '<i>None found</i>'}</div>
        </div>
        '''
    
    asyncio.ensure_future(api.create_window(src=html, name="Similarity Search Results"))

async def visualize_cells_interactive(
    original_image: List[Optional[np.ndarray]],
    segmentation_mask: Any,  # NOW: Accepts np.ndarray OR List[np.ndarray]
    cell_records: Optional[List[Dict[str, Any]]] = None
):
    """Create interactive HTML visualization with colored masks and hover tooltips showing cell metadata."""

    import random
    uid = f"viz_{random.randint(0,999999)}"
    mask_alpha = 0.3
    
    # NEW: Parse mask input - support multi-mask mode
    if isinstance(segmentation_mask, (list, tuple)):
        cell_mask_input = segmentation_mask[0]
        nucleus_mask_input = segmentation_mask[1] if len(segmentation_mask) > 1 else None
    else:
        cell_mask_input = segmentation_mask
        nucleus_mask_input = None
    
    # Ensure mask is numpy array
    if isinstance(cell_mask_input, str):
        mask_bytes = base64.b64decode(cell_mask_input)
        mask_img = PILImage.open(io.BytesIO(mask_bytes))
        segmentation_mask = np.array(mask_img)
    else:
        segmentation_mask = cell_mask_input
    
    # NEW: Convert nucleus mask if provided
    nucleus_mask = None
    if nucleus_mask_input is not None:
        if isinstance(nucleus_mask_input, str):
            mask_bytes = base64.b64decode(nucleus_mask_input)
            mask_img = PILImage.open(io.BytesIO(mask_bytes))
            nucleus_mask = np.array(mask_img)
        else:
            nucleus_mask = nucleus_mask_input
    
    # Determine image dimensions
    first = next((ch for ch in original_image if ch is not None), None)
    if first is None:
        raise ValueError("original_image has no valid channels")
    H, W = first.shape[:2]
    C = len(original_image)

    
    mask = segmentation_mask.astype(np.uint32)
    unique_ids = np.unique(mask[mask > 0])
    n_instances = len(unique_ids)
    
    # Use glasbey colormap - try colorcet first, fallback to tab20
    try:
        import colorcet
        colormap = colorcet.cm['glasbey']
    except (ImportError, KeyError):
        try:
            await micropip.install(["colorcet"])
            import colorcet
            colormap = colorcet.cm['glasbey']
        except:
            import matplotlib.colormaps as cmaps
            colormap = cmaps['tab20']
    
    # Create colored mask (RGBA)
    colored_mask = np.zeros((H, W, 4), dtype=np.uint8)
    for idx, instance_id in enumerate(unique_ids):
        if instance_id == 0:
            continue
        color_value = idx / max(n_instances - 1, 1) if n_instances > 1 else 0
        color_rgba = colormap(color_value)
        color_rgb = tuple(int(c * 255) for c in color_rgba[:3])
        colored_mask[mask == instance_id] = [*color_rgb, int(255 * mask_alpha)]
    
    # Convert mask to base64 PNG (much faster than JSON)
    mask_png = np_to_base64_png(colored_mask, normalize=False)
    
    # NEW: Create nucleus mask overlay if provided (cyan color)
    nucleus_png = None
    if nucleus_mask is not None:
        nucleus_mask = nucleus_mask.astype(np.uint32)
        nucleus_unique_ids = np.unique(nucleus_mask[nucleus_mask > 0])
        colored_nucleus_mask = np.zeros((H, W, 4), dtype=np.uint8)
        for nuc_id in nucleus_unique_ids:
            if nuc_id == 0:
                continue
            # Cyan color for nucleus boundaries with higher alpha
            colored_nucleus_mask[nucleus_mask == nuc_id] = [0, 255, 255, int(255 * 0.6)]
        nucleus_png = np_to_base64_png(colored_nucleus_mask, normalize=False)
    
    # Channel names and colors
    try:
        channel_names = fixed_channel_order[:C]
    except NameError:
        channel_names = [f'Channel {i}' for i in range(C)]
    
    # Short names for buttons
    channel_short_names = ['BF', '405', '488', '638', '561', '730']
    
    
    # Pre-render each channel as PNG (MUCH FASTER than .tolist())
    channel_images_b64 = {}
    available_channels = []
    
    for ch_idx in range(C):
        ch = original_image[ch_idx]
        if ch is None:
            continue
        if ch.max() == 0:
            continue
        available_channels.append(ch_idx)
        if ch.max() > 255:
            norm = (ch.astype(np.float64) / ch.max() * 255).astype(np.uint8)
        else:
            norm = ch.astype(np.uint8)
        if str(ch_idx) in color_map:
            r, g, b = color_map[str(ch_idx)]
            rgb_ch = np.stack([
                (norm * r).astype(np.uint8),
                (norm * g).astype(np.uint8),
                (norm * b).astype(np.uint8)
            ], axis=-1)
        else:
            rgb_ch = np.stack([norm, norm, norm], axis=-1)
        channel_images_b64[ch_idx] = np_to_base64_png(rgb_ch, normalize=False)

    
    # Build cell metadata lookup for hover (use index as cell identity)
    cell_meta_js = {}
    if cell_records:
        # Map directly from cell_records index to unique_ids
        for idx, cell in enumerate(cell_records):
            if idx < len(unique_ids):
                instance_id = unique_ids[idx]
                cell_meta_js[int(instance_id)] = {
                    "index": idx,
                    **{k: v for k, v in cell.items() 
                    if k not in ["image", "clip_embedding", "dino_embedding"]}
                }
    
    # Downsample mask for JS (every 4th pixel) - only small data now
    step = 4
    mask_small = mask[::step, ::step].tolist()
    
    # Build channel buttons HTML
    channel_buttons_html = ""
    for ch_idx in available_channels:
        short_name = channel_short_names[ch_idx] if ch_idx < len(channel_short_names) else f'Ch{ch_idx}'
        bg_color = "#007bff"  # All channels start active
        channel_buttons_html += f'''
            <button id="{uid}_ch{ch_idx}" onclick="toggleChannel({ch_idx})" 
                    style="padding:4px 12px;margin-right:4px;background:{bg_color};color:#fff;border:none;border-radius:3px;cursor:pointer;">
                {short_name}
            </button>
        '''
    
    # Build channel images object for JavaScript
    channel_imgs_js = {ch_idx: f"data:image/png;base64,{img_b64}" 
                       for ch_idx, img_b64 in channel_images_b64.items()}
    
    # Default: all channels active
    initial_active = available_channels
    
    html = f'''
    <div id="{uid}" style="font-family:Arial,sans-serif;">
        <div style="margin-bottom:8px;">
            {channel_buttons_html}
            <button id="{uid}_on" onclick="toggleMask(true)" 
                    style="padding:4px 12px;margin-right:4px;background:#007bff;color:#fff;border:none;border-radius:3px;cursor:pointer;">
                Mask ON
            </button>
            <button id="{uid}_off" onclick="toggleMask(false)" 
                    style="padding:4px 12px;background:#6c757d;color:#fff;border:none;border-radius:3px;cursor:pointer;">
                Mask OFF
            </button>
            {f"""
            <button id="{uid}_nuc_on" onclick="toggleNucleus(true)" 
                    style="padding:4px 12px;margin-left:8px;background:#17a2b8;color:#fff;border:none;border-radius:3px;cursor:pointer;">
                Nuclei ON
            </button>
            <button id="{uid}_nuc_off" onclick="toggleNucleus(false)" 
                    style="padding:4px 12px;background:#6c757d;color:#fff;border:none;border-radius:3px;cursor:pointer;">
                Nuclei OFF
            </button>
            """ if nucleus_png else ''}
        </div>
        <div style="position:relative;display:inline-block;">
            <canvas id="{uid}_canvas" width="{W}" height="{H}" style="max-width:500px;display:block;"></canvas>
            <div id="{uid}_tip" style="display:none;position:absolute;background:rgba(0,0,0,0.8);color:#fff;padding:6px 10px;border-radius:4px;font-size:11px;pointer-events:none;white-space:pre;z-index:10;"></div>
        </div>
    </div>
    <script>
    (function() {{
        const canvas = document.getElementById("{uid}_canvas");
        const ctx = canvas.getContext("2d");
        const tip = document.getElementById("{uid}_tip");
        const btnOn = document.getElementById("{uid}_on");
        const btnOff = document.getElementById("{uid}_off");
        const meta = {json.dumps(cell_meta_js)};
        const mask = {json.dumps(mask_small)};
        const step = {step};
        const W = {W};
        const H = {H};
        const hasNucleus = {json.dumps(nucleus_png is not None)};
        
        // Load channel images
        const channelImagesData = {json.dumps(channel_imgs_js)};
        const channelImages = {{}};
        const maskImg = new Image();
        const nucleusImg = hasNucleus ? new Image() : null;
        let imagesLoaded = 0;
        const totalImages = Object.keys(channelImagesData).length + 1 + (hasNucleus ? 1 : 0);
        
        // Load mask image
        maskImg.onload = () => {{
            imagesLoaded++;
            if (imagesLoaded === totalImages) updateImage();
        }};
        maskImg.src = "data:image/png;base64,{mask_png}";
        
        // Load nucleus image if available
        if (hasNucleus) {{
            nucleusImg.onload = () => {{
                imagesLoaded++;
                if (imagesLoaded === totalImages) updateImage();
            }};
            nucleusImg.src = "data:image/png;base64,{nucleus_png if nucleus_png else ''}";
        }}
        
        // Load channel images
        for (const [chIdx, imgSrc] of Object.entries(channelImagesData)) {{
            const img = new Image();
            img.onload = () => {{
                imagesLoaded++;
                if (imagesLoaded === totalImages) updateImage();
            }};
            img.src = imgSrc;
            channelImages[chIdx] = img;
        }}
        
        let activeChannels = {json.dumps(initial_active)};
        let showMask = true;
        let showNucleus = hasNucleus;
        
        window.toggleChannel = function(channelIdx) {{
            const idx = activeChannels.indexOf(channelIdx);
            if (idx === -1) {{
                activeChannels.push(channelIdx);
            }} else {{
                activeChannels.splice(idx, 1);
            }}
            updateImage();
            
            const btn = document.getElementById("{uid}_ch" + channelIdx);
            if (btn) {{
                btn.style.background = (idx === -1) ? "#007bff" : "#6c757d";
            }}
        }};
        
        window.toggleMask = function(on) {{
            showMask = on;
            updateImage();
            btnOn.style.background = on ? "#007bff" : "#6c757d";
            btnOff.style.background = on ? "#6c757d" : "#007bff";
        }};
        
        if (hasNucleus) {{
            const btnNucOn = document.getElementById("{uid}_nuc_on");
            const btnNucOff = document.getElementById("{uid}_nuc_off");
            
            window.toggleNucleus = function(on) {{
                showNucleus = on;
                updateImage();
                btnNucOn.style.background = on ? "#17a2b8" : "#6c757d";
                btnNucOff.style.background = on ? "#6c757d" : "#17a2b8";
            }};
        }}
        
        function updateImage() {{
            // Clear canvas
            ctx.clearRect(0, 0, W, H);
            
            // Use 'lighter' blend mode for additive compositing (GPU-accelerated!)
            ctx.globalCompositeOperation = 'lighter';
            
            // Draw all active channels with additive blending
            for (const chIdx of activeChannels) {{
                const img = channelImages[chIdx];
                if (img && img.complete) {{
                    ctx.drawImage(img, 0, 0, W, H);
                }}
            }}
            
            // Draw mask overlay
            if (showMask && maskImg.complete) {{
                ctx.globalCompositeOperation = 'source-over';
                ctx.drawImage(maskImg, 0, 0, W, H);
            }}
            
            // Draw nucleus overlay
            if (hasNucleus && showNucleus && nucleusImg && nucleusImg.complete) {{
                ctx.globalCompositeOperation = 'source-over';
                ctx.drawImage(nucleusImg, 0, 0, W, H);
            }}
        }}
        
        canvas.onmousemove = (e) => {{
        const rect = canvas.getBoundingClientRect();
        const scaleX = W / rect.width;
        const scaleY = H / rect.height;
        const x = Math.floor((e.clientX - rect.left) * scaleX);
        const y = Math.floor((e.clientY - rect.top) * scaleY);
        const mx = Math.floor(x / step), my = Math.floor(y / step);
        if (my >= 0 && my < mask.length && mx >= 0 && mx < mask[0].length) {{
            const id = mask[my][mx];
            if (id > 0 && meta[id]) {{
                const m = meta[id];
                let txt = `Cell ${{m.index}}`;
                
                // Morphological features
                if (m.area != null) txt += `\\nArea: ${{m.area.toFixed(1)}}`;
                if (m.perimeter != null) txt += `\\nPerimeter: ${{m.perimeter.toFixed(1)}}`;
                if (m.equivalent_diameter != null) txt += `\\nEq Diameter: ${{m.equivalent_diameter.toFixed(2)}}`;
                if (m.bbox_width != null) txt += `\\nBBox W: ${{m.bbox_width.toFixed(1)}}`;
                if (m.bbox_height != null) txt += `\\nBBox H: ${{m.bbox_height.toFixed(1)}}`;
                if (m.aspect_ratio != null) txt += `\\nAspect Ratio: ${{m.aspect_ratio.toFixed(3)}}`;
                if (m.circularity != null) txt += `\\nCircularity: ${{m.circularity.toFixed(3)}}`;
                if (m.eccentricity != null) txt += `\\nEccentricity: ${{m.eccentricity.toFixed(3)}}`;
                if (m.solidity != null) txt += `\\nSolidity: ${{m.solidity.toFixed(3)}}`;
                if (m.convexity != null) txt += `\\nConvexity: ${{m.convexity.toFixed(3)}}`;
                
                // Texture features
                if (m.brightness != null) txt += `\\nBrightness: ${{m.brightness.toFixed(3)}}`;
                
                // Fluorescence intensity features - show ALL available fields
                for (const key in m) {{
                    // Cell intensities
                    if (key.startsWith('mean_intensity_') && key.endsWith('_cell') && m[key] != null) {{
                        const channelName = key.replace('mean_intensity_', '').replace('_cell', '').replace(/_/g, ' ');
                        txt += `\\n${{channelName}} (cell): ${{m[key].toFixed(1)}}`;
                    }}
                    // Nucleus intensities
                    if (key.startsWith('mean_intensity_') && key.endsWith('_nucleus') && m[key] != null) {{
                        const channelName = key.replace('mean_intensity_', '').replace('_nucleus', '').replace(/_/g, ' ');
                        txt += `\\n${{channelName}} (nuc): ${{m[key].toFixed(1)}}`;
                    }}
                    // Cytosol intensities
                    if (key.startsWith('mean_intensity_') && key.endsWith('_cytosol') && m[key] != null) {{
                        const channelName = key.replace('mean_intensity_', '').replace('_cytosol', '').replace(/_/g, ' ');
                        txt += `\\n${{channelName}} (cyto): ${{m[key].toFixed(1)}}`;
                    }}
                    // Ratios
                    if (key.startsWith('ratio_') && key.endsWith('_nuc_cyto') && m[key] != null) {{
                        const channelName = key.replace('ratio_', '').replace('_nuc_cyto', '').replace(/_/g, ' ');
                        txt += `\\n${{channelName}} (N/C): ${{m[key].toFixed(2)}}`;
                    }}
                    // Top10 intensities (backward compatible)
                    if (key.startsWith('top10_mean_intensity_') && m[key] != null) {{
                        const channelName = key.replace('top10_mean_intensity_', '').replace(/_/g, ' ');
                        txt += `\\nTop 10% ${{channelName}}: ${{m[key].toFixed(1)}}`;
                    }}
                }}
                
                tip.textContent = txt;
                tip.style.display = "block";
                tip.style.left = (e.clientX - rect.left + 10) + "px";
                tip.style.top = (e.clientY - rect.top + 10) + "px";
                return;
            }}
        }}
        tip.style.display = "none";
    }};
    canvas.onmouseleave = () => {{ tip.style.display = "none"; }};
    }})();
    </script>
    '''
    asyncio.ensure_future(api.create_window(src=html, name="Image Segmentation"))

import pandas as pd
import matplotlib.pyplot as plt

def show_matplotlib_fig(fig, name: str, dpi: int = 200, max_width: str = "100%") -> None:
    """Display matplotlib figure via asyncio.ensure_future(api.create_window()). DO NOT use plt.show()."""
    buf = BytesIO()
    fig.savefig(buf, format="png", dpi=dpi, bbox_inches="tight")
    plt.close(fig)
    b64 = base64.b64encode(buf.getvalue()).decode("utf-8")
    asyncio.ensure_future(api.create_window(
        src=(
            f'<div style="text-align:center;">'
            f'<img src="data:image/png;base64,{b64}" style="max-width:{max_width};"/>'
            f"</div>"
        ),
        name=name
    ))

# System Prompts, Workflow Templates, and Utility Functions

# Build DataFrame utility
def build_df_from_records(
    records: List[Dict],
    *,
    distance_key: str = "distance_from_center",
    well_id_key: str = "well_id",
    fallback_well_from_position: bool = True,
    exclude_keys: Optional[List[str]] = None,
) -> pd.DataFrame:
    """Convert cell_records to DataFrame with ALL available data fields."""
    default_exclude = {
        "image",
        "clip_embedding",
        "dino_embedding",
    }
    
    if exclude_keys:
        default_exclude.update(exclude_keys)
    
    def _safe_float(v):
        try:
            if v is None:
                return np.nan
            return float(v)
        except Exception:
            return np.nan
    
    def _safe_value(v):
        if v is None:
            return None
        if isinstance(v, (int, float, str, bool)):
            return v
        if isinstance(v, dict):
            return str(v)
        if isinstance(v, (list, tuple)):
            if len(v) < 10:
                return str(v)
            return None
        return str(v)

    rows = []
    for idx, c in enumerate(records):
        pos = c.get("position", {}) or {}

        wid = c.get(well_id_key, None)
        if (wid is None) and fallback_well_from_position:
            wr = pos.get("well_row", None)
            wc = pos.get("well_col", None)
            if (wr is not None) and (wc is not None):
                wid = f"{wr}{wc}"
        if wid is None:
            wid = "well_1"

        d = _safe_float(c.get(distance_key, np.nan))

        row = {
            "cell_index": idx,
            "well_id": wid,
            "distance_mm": d,
        }
        
        if pos:
            for pos_key, pos_val in pos.items():
                if isinstance(pos_val, (int, float)):
                    row[pos_key] = _safe_float(pos_val)
                else:
                    row[pos_key] = pos_val

        for key, value in c.items():
            if key in default_exclude:
                continue
            if key == "position":
                continue
            if key == well_id_key and key in row:
                continue
            
            if any(substring in key.lower() for substring in [
                "intensity", "area", "perimeter", "diameter", "width", "height",
                "ratio", "circularity", "eccentricity", "solidity", "convexity",
                "brightness", "contrast", "homogeneity", "energy", "correlation",
                "similarity", "score", "distance", "index"
            ]):
                row[key] = _safe_float(value)
            else:
                row[key] = _safe_value(value)

        rows.append(row)

    df = pd.DataFrame(rows)

    if distance_key in df.columns or "distance_mm" in df.columns:
        df = df[np.isfinite(df["distance_mm"])].copy()

    return df


# System Prompts
SYSTEM_PROMPT_1 = """
You are an AI microscope-control agent for Agent-Lens with Vector Database integration.

PRIMARY ROLE
- Drive the microscope: navigate to wells, autofocus, move to positions (FoVs), snap images (BF + fluorescence), and run scans.
- Do on-the-fly analysis: segmentation, single-cell metadata extraction, similarity search, basic plotting/visualization as requested.
- variable 'microscope' is the microscope object, 'agent_lens_service' is the agent lens service object. Other variables are defined in the environment already(color_map, fixed_channel_order, microscope_id, etc).

STRICT OUTPUT RULES
- Respond with ONLY executable Python code for this Jupyter notebook (no prose).
- Respond with direct code, do not wrap code in functions or classes.
- Any figure/image output MUST be displayed using asyncio.ensure_future(api.create_window(src=html, name=name)) or pre-defined display functions.
- DO NOT use 'await api.create_window()' - it will block execution.
- DO NOT rely on plt.show() or Display().
- For microscope actions, use the provided async tools (await microscope.* / await snap_image / await segment_image / await agent_lens_service.build_cell_records / etc).
- Write a linear, top-to-bottom notebook script. DO NOT define new functions or classes unless necessary.
- Minimize stdout. Do NOT print large objects (images/arrays, full records lists, full tool schemas, full microscope status dicts).
- Only print short progress messages (1 line) and small scalar summaries.

WORKING STYLE
- Default to minimal steps that answer the user's request.
- Reuse existing variables and helper functions already defined in the notebook.
- If a request depends on missing prerequisites, create the needed data first, then proceed.

DATA CONVENTIONS
- color_map is a dict of channel number to RGB tuple (0-255, 0-255, 0-255)
- - default 
color_map = {
    "0": (1.0, 1.0, 1.0),  # BF: gray
    "1": (0.0, 0.0, 1.0),  # 405nm: blue
    "2": (0.0, 1.0, 0.0),  # 488nm: green
    "3": (1.0, 0.0, 0.0),  # 638nm: red
    "4": (1.0, 1.0, 0.0),  # 561nm: yellow(simulated)
}
You can update color_map based on user instructions
- raw_image is multi-channel, list of numpy arrays for different channel.
- raw_image and norm_image are always the same shape and channels order. 0: BF, 1: 405nm, 2: 488nm, 3: 638nm, 4: 561nm
- cell_records are lists of dicts with fields:
  - uuid: Unique cell identifier
  - image: merged composite image (50x50). Note: This is empty, you need to use 'agent_lens_service.fetch_cell_data()' to get complete cell data, which includes the image.
  - morphology: area, aspect_ratio, circularity, solidity, eccentricity...
  - position info: 'position[x]', 'position[y]', 'distance_from_center', 'well_id'
  - optional image crops: 'image' (merged composite)
  - cell_records intensity features:
    * Single mask mode: mean_intensity_<channel>_cell, top10_mean_intensity_<channel>
    * Multi-mask mode (when nucleus mask provided):
        - mean_intensity_<channel>_cell
        - mean_intensity_<channel>_nucleus  
        - mean_intensity_<channel>_cytosol
        - ratio_<channel>_nuc_cyto (nucleus-to-cytosol ratio)

VECTOR DATABASE SIMILARITY SEARCH
- Use similarity_search_with_filters() for backward-compatible filtering
- Or use agent_lens_service.similarity_search_cells() directly for vector database native search
- Supports native metadata filtering using vector database where clause syntax
- Returns cells with similarity_score (0-1, higher is more similar)

ANALYSIS CAPABILITIES YOU MUST SUPPORT (WHEN ASKED)
- Quick inspection: show raw/normalized images; show segmentation overlay; interactive cell viewer (visualize_cells_interactive).
- Similarity search: user selects query cell(s) from cell_records; run similarity_search_with_filters or similarity_search_cells; show_similarity_results.
- Spatial analysis: plot cell density, morphology metrics, or custom classifications vs distance from well center.
- Custom cell classification: user can define subpopulations based on any combination of intensity/morphology features.
- Multi-well statistics: aggregate metrics across wells with mean±SEM error bars.
- Intensity metric policy: When gating by fluorescence, compute both mean and top10 metrics if available.

DISPLAY REQUIREMENT
- Any matplotlib figure must be encoded to html and shown via api.create_window(src=html, name=name). # no await

DEFAULT SAFETY / HYGIENE
- Validate required keys/columns before use; if missing, fall back gracefully.
- Close matplotlib figures after saving/encoding.
"""
print(SYSTEM_PROMPT_1)

print("---")
print(f"The microscope has the following channels: {fixed_channel_order}")
print("---")
# Print available tools
print("\n" + "="*80)
print("AVAILABLE FUNCTIONS:")
print("="*80)
#print micrscope schema_tool decorator
microscope_tools = [json.dumps(tool.__schema__, indent=2) for tool in microscope.values() if callable(tool)]
print(f"This microscope has the following tools: {microscope_tools}")

agent_lens_tools = [json.dumps(tool.__schema__, indent=2) for tool in agent_lens_service.values() if callable(tool)]
print(f"Agent Lens service has the following tools: {agent_lens_tools}")

def print_tool_doc(func):
    print("---")
    print(f"Function: {func.__name__}")
    print(func.__doc__)
    print("---")


print_tool_doc(snap_image)
print_tool_doc(percentile_normalize)
print_tool_doc(overlay)
print_tool_doc(segment_image)
print_tool_doc(make_stage_offsets)
print_tool_doc(similarity_search_with_filters)
print_tool_doc(show_similarity_results)
print_tool_doc(show_matplotlib_fig)
print_tool_doc(build_df_from_records)
print_tool_doc(wait_for_snap_segment_extract)

# Workflow Templates
SYSTEM_PROMPT_2 = """
WORKFLOW TEMPLATES

----------------------------------------------------------------------
0) TAKE A LOOK (Navigate → Focus → Snap → Segment → View)
----------------------------------------------------------------------

await microscope.navigate_to_well('B', 2, well_plate_type='96')
await microscope.reflection_autofocus()

# If user asked for Brightfield, 488nm, 561nm channels
channel_config = [
  {"channel": "BF_LED_matrix_full", "exposure_time": 10, "intensity": 20},
  {"channel": "Fluorescence_488_nm_Ex", "exposure_time": 100, "intensity": 60},
  {"channel": "Fluorescence_561_nm_Ex", "exposure_time": 100, "intensity": 60},
]

raw_image = await snap_image(channel_config)
norm_image = percentile_normalize(raw_image) # The channels order is always the same as fixed_channel_order
seg_mask = await segment_image(norm_image)

# If nucleus channel is available, segment nuclei
# nucleus_mask = await segment_image(norm_image[1])  # If user told you 405nm channel is for nucleus, 405nm is always the SECOND channel of 'fixed_channel_order' and 'norm_image', will not changed by channel_config

status = await microscope.get_status()
cell_records = await agent_lens_service.build_cell_records(
    raw_image, seg_mask, status, application_id="hypha-agents-notebook", color_map=color_map
)
# If nucleus channel is available, segment nuclei
# cell_records = await agent_lens_service.build_cell_records(
#     raw_image, 
#     [cell_mask, nucleus_mask],  # Pass as list for multi-mask mode
#     status, 
#     application_id="hypha-agents-notebook",
#     color_map=color_map
# )

await visualize_cells_interactive(
  original_image=norm_image,
  segmentation_mask=seg_mask,
  cell_records=cell_records,
)
# If nucleus channel is available, visualize nuclei also
# await visualize_cells_interactive(
#   original_image=norm_image,
#   segmentation_mask=[cell_mask, nucleus_mask],
#   cell_records=cell_records,
# )

print(f"Found {len(cell_records)} cells")

----------------------------------------------------------------------
1) FIND SIMILAR CELLS (Backward-Co
patible API)
----------------------------------------------------------------------

# Extract query cell
query_cell_indices = [81]
query_cell_records = [cell_records[i] for i in query_cell_indices if i < len(cell_records)]

# Old-style filter configuration (backward compatible)
relative_config = {
    "size_tolerance": 0.4,
    "circularity_tol": 0.15,
    "eccentricity_tol": 0.15,
    "solidity_tol": 0.1,
    "aspect_ratio_tol": 0.3,
}

range_config = {} # Default is empty, no range filtering
# Example if range is needed
# range_config = {"top10_mean_intensity_Fluorescence_488_nm_Ex_cell": {"min": 15}}



similarity_config = {
    "final_score_threshold": 0.7,
}

# Use backward-compatible function
similar_cells = await similarity_search_with_filters(
    query_cell_records=query_cell_records,
    relative_config=relative_config,
    range_config=range_config,
    similarity_config=similarity_config,
    n_results=100
)

print(f"Found {len(similar_cells)} similar cells")
show_similarity_results(query_cell_records, similar_cells, max_examples=20)

----------------------------------------------------------------------
1b) FIND SIMILAR CELLS (Direct Vector Database API)
----------------------------------------------------------------------

# Extract query UUIDs
query_uuids = [cell["uuid"] for cell in query_cell_records]

# Direct Vector Database where clause
similar_cells = await agent_lens_service.similarity_search_cells(
    query_cell_uuids=query_uuids,
    application_id="hypha-agents-notebook",
    n_results=100,
    metadata_filters={
        "$and": [
            {"area": {"$gt": 200, "$lt": 3000}},
            {"circularity": {"$gte": 0.7}},
            {"top10_mean_intensity_Fluorescence_488_nm_Ex_cell": {"$gte": 15}}
        ]
    },
    similarity_threshold=0.7
)

show_similarity_results(query_cell_records, similar_cells, max_examples=20)

----------------------------------------------------------------------
2) SCAN WITH SIMILARITY SEARCH
----------------------------------------------------------------------

rows = ("B", "C", "D")
cols = tuple(range(2, 5))
wells = [f"{row}{col}" for row in rows for col in cols]

grid_size = 3
well_offsets = make_stage_offsets(grid_size=grid_size)

channel_config = [
    {"channel": "Fluorescence_488_nm_Ex", "exposure_time": 100, "intensity": 60},
    {"channel": "Fluorescence_561_nm_Ex", "exposure_time": 100, "intensity": 60},
    {"channel": "BF_LED_matrix_full", "exposure_time": 10, "intensity": 20},

]

all_cell_records = []
resp = await agent_lens_service.snap_segment_extract_put_queue(
    microscope_id=microscope_id,
    channel_config=channel_config,
    application_id=application_id,
    scale=8,
    wells=wells,
    well_offset=well_offset,
    well_plate_type="96",
    nucleus_channel_name="Fluorescence_405_nm_Ex", # If user TOLD you 405nm channel is for nucleus, otherwise None
    color_map=color_map,
)
print(f"Queued {len(wells)} wells with {len(well_offset)} positions each")
print(f"Total FoVs: {len(wells) * len(well_offset)} (queue size={resp['queue_size']})")
print(f"Queuing took {time.time()-t0:.2f}s")

# Wait for all jobs to complete
print('Waiting for all snap+segment+extract jobs to finish...')
result_cell_records = await wait_for_snap_segment_extract(agent_lens_service)
all_cell_records.extend(result_cell_records)

print(f'Total cells extracted: {len(all_cell_records)}')

# Now search for similar cells
similar_cells = await similarity_search_with_filters(
    query_cell_records=query_cell_records,
    relative_config=relative_config,
    range_config=range_config,
    similarity_config=similarity_config,
    n_results=100
)

show_similarity_results(query_cell_records, similar_cells, max_examples=20)

----------------------------------------------------------------------
3) SPATIAL ANALYSIS
----------------------------------------------------------------------

df = build_df_from_records(all_scanned_cell_records)
metric = "area"

bin_w = 0.25
bins = np.arange(0, df["distance_mm"].max() + bin_w, bin_w)
bin_centers = 0.5 * (bins[:-1] + bins[1:])

df["r_bin"] = pd.cut(df["distance_mm"], bins=bins, include_lowest=True)

per_well = (
  df.groupby(["well_id", "r_bin"])[metric]
    .mean()
    .unstack("r_bin")
)

mean_curve = per_well.mean(axis=0).to_numpy()
sem_curve = per_well.sem(axis=0, ddof=1).to_numpy()

fig, ax = plt.subplots(figsize=(5, 4))
ax.errorbar(bin_centers, mean_curve, yerr=sem_curve, marker="o", linewidth=2, capsize=2)
ax.set_xlabel("Distance from well center (mm)")
ax.set_ylabel(metric)
ax.grid(True, alpha=0.25, linestyle="--")
show_matplotlib_fig(fig, name="Spatial Analysis")

REMINDER
- Use show_matplotlib_fig(fig, name=name) to display plots
- Use build_df_from_records() for DataFrame creation
- Use similarity_search_with_filters() for backward compatibility
"""


print(SYSTEM_PROMPT_2)

print("\n" + "="*80)
print("✓ All functions and templates loaded!")
print("✓ Ready for AI agent control")
print("="*80)


You are an AI microscope-control agent for Agent-Lens with Vector Database integration.

PRIMARY ROLE
- Drive the microscope: navigate to wells, autofocus, move to positions (FoVs), snap images (BF + fluorescence), and run scans.
- Do on-the-fly analysis: segmentation, single-cell metadata extraction, similarity search, basic plotting/visualization as requested.
- variable 'microscope' is the microscope object, 'agent_lens_service' is the agent lens service object. Other variables are defined in the environment already(color_map, fixed_channel_order, microscope_id, etc).

STRICT OUTPUT RULES
- Respond with ONLY executable Python code for this Jupyter notebook (no prose).
- Respond with direct code, do not wrap code in functions or classes.
- Any figure/image output MUST be displayed using api.create_window(src=html, name=name) or pre-defined display functions.
- DO NOT use 'await api.create_window()' - it will block execution.
- DO NOT rely on plt.show() or Display().
- For microscope

switch sample to HPA 1 simulated sample dataset

In [2651]:
import asyncio

print('Listing available simulation samples...')
samples = await microscope.list_simulation_samples()
print('Available samples keys:', list(samples.keys()) if isinstance(samples, dict) else type(samples))

print('Switching sample to HPA dataset (HPA_PLATE1)...')
active = await microscope.switch_sample('HPA_PLATE1')
print('Active sample:', active.get('active_sample', active))

Listing available simulation samples...
Available samples keys: ['U2OS_FUCCI', 'HPA_PLATE1', 'HPA_PLATE2', 'HPA_PLATE3', 'HPA_PLATE4', 'HPA_PLATE5']
Switching sample to HPA dataset (HPA_PLATE1)...
Active sample: HPA_PLATE1


Take image at well A1,(1 Fov per well) (only 405, 488, 561, 638 nm channel avaliable). Segment cells and nucleus(channel is 405nm) and show me

In [2652]:
import numpy as np

print('Navigating to well A1 and autofocusing...')
await microscope.navigate_to_well('A', 1, well_plate_type='96')
await microscope.reflection_autofocus()

print('Snapping 4 fluorescence channels (405, 488, 561, 638 nm)...')
channel_config = [
    {"channel": "Fluorescence_405_nm_Ex", "exposure_time": 100, "intensity": 60},
    {"channel": "Fluorescence_488_nm_Ex", "exposure_time": 100, "intensity": 60},
    {"channel": "Fluorescence_561_nm_Ex", "exposure_time": 100, "intensity": 60},
    {"channel": "Fluorescence_638_nm_Ex", "exposure_time": 100, "intensity": 60},
]

raw_image = await snap_image(channel_config)

print('Normalizing channels...')
norm_image = percentile_normalize(raw_image)

print('Segmenting cells from composite image...')
cell_mask = await segment_image(norm_image)

print('Segmenting nuclei from 405 nm channel...')
# 405 nm is the second channel in fixed_channel_order/norm_image
nucleus_mask = await segment_image(norm_image[1])

print('Building cell records with cell + nucleus masks...')
status = await microscope.get_status()
cell_records = await agent_lens_service.build_cell_records(
    raw_image,
    [cell_mask, nucleus_mask],
    status,
    application_id="hypha-agents-notebook",
    color_map=color_map,
)

print(f'Found {len(cell_records)} cells with nucleus segmentation')

print('Launching interactive cell + nucleus viewer...')
await visualize_cells_interactive(
    original_image=norm_image,
    segmentation_mask=[cell_mask, nucleus_mask],
    cell_records=cell_records,
)


Navigating to well A1 and autofocusing...
Snapping 4 fluorescence channels (405, 488, 561, 638 nm)...
Normalizing channels...
Segmenting cells from composite image...
Segmenting nuclei from 405 nm channel...
Building cell records with cell + nucleus masks...
Found 215 cells with nucleus segmentation
Launching interactive cell + nucleus viewer...


Please plot the 638nm channel distribution for cells

In [2653]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print('Building DataFrame from existing cell_records...')
df = build_df_from_records(cell_records)

col_mean = 'mean_intensity_Fluorescence_638_nm_Ex_cell'
col_top10 = 'top10_mean_intensity_Fluorescence_638_nm_Ex_cell'

available_cols = [c for c in [col_mean, col_top10] if c in df.columns]
print('Available 638 nm intensity columns:', available_cols)

fig, ax = plt.subplots(figsize=(5, 4))

if col_mean in df.columns:
    ax.hist(df[col_mean].dropna(), bins=40, alpha=0.6, label='Mean (cell)', color='tab:blue')
if col_top10 in df.columns:
    ax.hist(df[col_top10].dropna(), bins=40, alpha=0.6, label='Top 10% (cell)', color='tab:orange')

ax.set_xlabel('638 nm intensity (a.u.)')
ax.set_ylabel('Cell count')
ax.set_title('Distribution of 638 nm channel intensity per cell')
ax.legend()
ax.grid(alpha=0.3, linestyle='--')

show_matplotlib_fig(fig, name='638nm_intensity_distribution')
plt.close(fig)


Building DataFrame from existing cell_records...
Available 638 nm intensity columns: ['mean_intensity_Fluorescence_638_nm_Ex_cell']


The 638nm is the marker of virus infection. I took a look and I think if the cell's mean intensity 638nm if over 15, we can mark it infected cells. Please make infected (I) vs non-infected (NI) violin plots of the 488 “target protein” intensity, make 3 gourp of plot to compare nucleus, cytosol, and whole cell's instesity

In [2654]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print('Preparing DataFrame and infection labels...')

df = build_df_from_records(cell_records)

threshold = 15.0
marker_col = 'mean_intensity_Fluorescence_638_nm_Ex_cell'

if marker_col not in df.columns:
    print('Error: 638 nm mean intensity column not found in DataFrame.')
else:
    df = df.copy()
    df['infection_status'] = np.where(df[marker_col] > threshold, 'I', 'NI')

    nucleus_col = 'mean_intensity_Fluorescence_488_nm_Ex_nucleus'
    cyto_col = 'mean_intensity_Fluorescence_488_nm_Ex_cytosol'
    cell_col = 'mean_intensity_Fluorescence_488_nm_Ex_cell'

    cols_available = [c for c in [nucleus_col, cyto_col, cell_col] if c in df.columns]
    print('Available 488 nm intensity columns:', cols_available)

    groups = [
        ('Nucleus', nucleus_col),
        ('Cytosol', cyto_col),
        ('Whole cell', cell_col),
    ]

    for title, col in groups:
        if col not in df.columns:
            print(f'Skipping {title} - column {col} not found')
            continue

        data_I = df.loc[df['infection_status'] == 'I', col].dropna()
        data_NI = df.loc[df['infection_status'] == 'NI', col].dropna()

        if len(data_I) == 0 or len(data_NI) == 0:
            print(f'Skipping {title} - insufficient data (I={len(data_I)}, NI={len(data_NI)})')
            continue

        fig, ax = plt.subplots(figsize=(4, 4))
        parts = ax.violinplot([data_NI, data_I], positions=[1, 2], showmeans=True, showextrema=True, widths=0.8)

        for i, pc in enumerate(parts['bodies']):
            pc.set_facecolor(['tab:blue', 'tab:red'][i])
            pc.set_edgecolor('black')
            pc.set_alpha(0.7)

        ax.set_xticks([1, 2])
        ax.set_xticklabels(['NI', 'I'])
        ax.set_ylabel('488 nm intensity (a.u.)')
        ax.set_title(f'488 nm target protein intensity - {title}')
        ax.grid(alpha=0.3, linestyle='--')

        show_matplotlib_fig(fig, name=f'488_violin_{title.replace(" ", "_").lower()}')
        plt.close(fig)


Preparing DataFrame and infection labels...
Available 488 nm intensity columns: ['mean_intensity_Fluorescence_488_nm_Ex_nucleus', 'mean_intensity_Fluorescence_488_nm_Ex_cytosol', 'mean_intensity_Fluorescence_488_nm_Ex_cell']


 Scan wells A1–A12,B1-B12...H1-H11 with 405/488/561/638. For each well, show 10 infected cell and 10 non infected cells' 488 image in green color.Add buttons for each well, so I can assign localization labels for NI and I populations. After annotation, generate an interactive summary table and a Circos plot of NI→I localization changes. Use api.create_window() for display.

localization labels:
Negative,
Nucleoplasm,
Nucleoli,
Nuclear membrane,
Cytosol,
Endoplasmic reticulum,
Golgi Apparatus,
Vesicles,
Plasma membrane,
Microtubules,
Mitochondria,
Nuclear speckles,
SCV(SCV= Single Cell Variation)

In [2663]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import json
from itertools import product

print('Defining wells to scan...')
rows = [chr(ord('A') + i) for i in range(8)]  # A-H
cols = list(range(1, 13))  # 1-12
wells = [f"{r}{c}" for r, c in product(rows, cols)]
print('Total wells:', len(wells))

print('Defining grid: 1 FoV per well at center (offset 0,0)...')
well_offset = [{"dx": 0.0, "dy": 0.0}]

channel_config = [
    {"channel": "Fluorescence_405_nm_Ex", "exposure_time": 100, "intensity": 60},
    {"channel": "Fluorescence_488_nm_Ex", "exposure_time": 100, "intensity": 60},
    {"channel": "Fluorescence_561_nm_Ex", "exposure_time": 100, "intensity": 60},
    {"channel": "Fluorescence_638_nm_Ex", "exposure_time": 100, "intensity": 60},
]

application_id = "hypha-agents-notebook"

print('Queuing snap+segment+extract jobs for all wells...')
start_time = time.time()
queue_resp = await agent_lens_service.snap_segment_extract_put_queue(
    microscope_id=microscope_id,
    channel_config=channel_config,
    application_id=application_id,
    scale=8,
    wells=wells,
    well_offset=well_offset,
    well_plate_type="96",
    nucleus_channel_name="Fluorescence_405_nm_Ex",
    color_map=color_map,
)
print('Queue response:', {k: queue_resp[k] for k in queue_resp if k != 'details'})

print('Waiting for all jobs to finish...')
all_cell_records = await wait_for_snap_segment_extract(agent_lens_service)
print('Total cells from all wells:', len(all_cell_records))

print('Building DataFrame from all cell records...')
df_all = build_df_from_records(all_cell_records)

marker_col = 'mean_intensity_Fluorescence_638_nm_Ex_cell'
infect_thresh = 15.0

if marker_col not in df_all.columns:
    print('Error: 638 nm mean intensity column missing; cannot classify infection.')
else:
    df_all['infection_status'] = np.where(df_all[marker_col] > infect_thresh, 'I', 'NI')

print('Sampling 10 I and 10 NI cells per well where available...')

col_488_cell = 'mean_intensity_Fluorescence_488_nm_Ex_cell'

sampled_cells = []
for well in wells:
    sub = df_all[df_all['well_id'] == well]
    if 'infection_status' not in sub.columns or sub.empty:
        continue
    sub_I = sub[sub['infection_status'] == 'I']
    sub_NI = sub[sub['infection_status'] == 'NI']
    if len(sub_I) == 0 and len(sub_NI) == 0:
        continue
    sample_I = sub_I.sample(n=min(10, len(sub_I)), random_state=0) if len(sub_I) > 0 else sub_I
    sample_NI = sub_NI.sample(n=min(10, len(sub_NI)), random_state=0) if len(sub_NI) > 0 else sub_NI
    sampled_cells.append(pd.concat([sample_I, sample_NI], axis=0))

if len(sampled_cells) == 0:
    print('No cells available for sampling.')
else:
    df_sampled = pd.concat(sampled_cells, axis=0)
    print('Total sampled cells:', len(df_sampled))

    uuids_sampled = df_sampled['uuid'].tolist()
    print('Fetching full cell data (including images) for sampled cells...')
    fetched_cells = await agent_lens_service.fetch_cell_data(
        uuids=uuids_sampled,
        application_id=application_id,
    )

    uuid_to_cell = {c['uuid']: c for c in fetched_cells}

    import base64
    from io import BytesIO
    from PIL import Image

    print('Preparing per-well annotation interface HTML...')

    localization_labels = [
        'Negative',
        'Nucleoplasm',
        'Nucleoli',
        'Nuclear membrane',
        'Cytosol',
        'Endoplasmic reticulum',
        'Golgi Apparatus',
        'Vesicles',
        'Plasma membrane',
        'Microtubules',
        'Mitochondria',
        'Nuclear speckles',
        'SCV(SCV= Single Cell Variation)',
    ]

    per_well_html_sections = []

    for well in wells:
        sub = df_sampled[df_sampled['well_id'] == well]
        if sub.empty:
            continue
        rows_html = []
        for _, row in sub.iterrows():
            uuid = row['uuid']
            cell_data = uuid_to_cell.get(uuid)
            if cell_data is None:
                continue
            img_b64 = cell_data.get('channel_488_image') or cell_data.get('image')
            if img_b64 is None:
                continue
            img_src = f"data:image/png;base64,{img_b64}"
            inf_status = row['infection_status']
            row_id = f"{well}_{inf_status}_{uuid}"

            options_html = ''.join([
                f'<option value="{label}">{label}</option>' for label in localization_labels
            ])

            rows_html.append(f'''
                <div style="display:flex; align-items:center; margin:4px 0;">
                    <img src="{img_src}" style="width:50px;height:50px;border:1px solid #ccc;margin-right:8px;" />
                    <span style="width:40px;display:inline-block;">{inf_status}</span>
                    <select id="loc_{row_id}" data-well="{well}" data-status="{inf_status}">
                        <option value="">(select localization)</option>
                        {options_html}
                    </select>
                </div>
            ''')

        if not rows_html:
            continue

        well_block = f'''
        <div style="border:1px solid #999;padding:8px;margin:8px;">
            <h3>Well {well}</h3>
            {''.join(rows_html)}
        </div>
        '''
        per_well_html_sections.append(well_block)

    all_wells_html = ''.join(per_well_html_sections)

    annotation_html = f'''
    <html>
    <head>
    <meta charset="utf-8" />
    <title>Per-well NI/I 488 nm localization annotation</title>
    </head>
    <body>
        <h2>Assign localization labels for NI and I populations (488 nm in green)</h2>
        <p>For each cell image, choose a localization label. Use the infection status tags (NI/I) shown next to images.</p>
        <div style="max-height:70vh;overflow:auto;border:1px solid #ccc;padding:4px;">
            {all_wells_html}
        </div>
        <button id="export_annotations" style="margin-top:10px;">Export annotations to console</button>
        <pre id="annotation_output" style="white-space:pre-wrap;font-size:10px;"></pre>
        <script>
            document.getElementById('export_annotations').onclick = function() {{
                var selects = document.querySelectorAll('select[id^="loc_"]');
                var data = [];
                selects.forEach(function(sel) {{
                    var val = sel.value;
                    if (!val) return;
                    var id = sel.id.substring(4);
                    var well = sel.getAttribute('data-well');
                    var status = sel.getAttribute('data-status');
                    data.push({{id: id, well: well, status: status, localization: val}});
                }});
                document.getElementById('annotation_output').textContent = JSON.stringify(data, null, 2);
                console.log('ANNOTATIONS_JSON_START');
                console.log(JSON.stringify(data));
                console.log('ANNOTATIONS_JSON_END');
            }};
        </script>
    </body>
    </html>
    '''

    api.create_window(src=annotation_html, name='NI_I_localization_annotation')

    print('Annotation interface launched. After you finish labeling, copy the JSON from the panel or browser console, then paste it into the notebook for further analysis/plots.')


Defining wells to scan...
Total wells: 96
Defining grid: 1 FoV per well at center (offset 0,0)...
Queuing snap+segment+extract jobs for all wells...
Queue response: {'success': True, 'queued': True, 'queue_size': 1}
Waiting for all jobs to finish...
Waiting for all snap+segment+extract jobs to finish...
[3.2s] Processing... queues[snap:0, seg:0, build:0], workers[snap:BUSY], results:0
[36.2s] Processing... queues[snap:0, seg:2, build:0], workers[snap:BUSY], results:215
[69.5s] Processing... queues[snap:0, seg:1, build:0], workers[snap:BUSY, seg/build:BUSY], results:793
[102.5s] Processing... queues[snap:0, seg:0, build:0], workers[snap:BUSY, seg/build:BUSY], results:1344
[140.8s] Processing... queues[snap:0, seg:0, build:1], workers[snap:BUSY, seg/build:BUSY], results:1774
[173.8s] Processing... queues[snap:0, seg:1, build:0], workers[snap:BUSY, seg/build:BUSY], results:2345
[207.1s] Processing... queues[snap:0, seg:1, build:0], workers[snap:BUSY, seg/build:BUSY], results:2921
[240.2s]

The UI looks great, there should be one option panel for a well, not a cell. also the options should be multi choice option, not single option. At the bottom, you should generate Summary table & Circos-style Plot
The style of Circos plot, showing the protein subcellular location of translocated proteins from non-infected cells (left) and infected cells (right)

In [ ]:
import json

print('Regenerating UI with correct 488-channel images and multi-select localization...')

# Rebuild sampledCells with explicit 488-channel images from previously fetched_cells
# We already had df_sampled and fetched_cells / uuid_to_cell from the scanning step

js_sampled = []
for _, row in df_sampled.iterrows():
    uid = row['uuid']
    cell_data = uuid_to_cell.get(uid, {})
    img_b64 = cell_data.get('channel_488_image') or cell_data.get('image')
    if not img_b64:
        continue
    js_sampled.append({
        'uuid': uid,
        'well_id': row['well_id'],
        'infection_status': row['infection_status'],
        'image': img_b64,
    })

print('Sampled cells with images:', len(js_sampled))

localization_labels = [
    'Negative',
    'Nucleoplasm',
    'Nucleoli',
    'Nuclear membrane',
    'Cytosol',
    'Endoplasmic reticulum',
    'Golgi Apparatus',
    'Vesicles',
    'Plasma membrane',
    'Microtubules',
    'Mitochondria',
    'Nuclear speckles',
    'SCV(SCV= Single Cell Variation)'
]

data_json = json.dumps(js_sampled)
labels_json = json.dumps(localization_labels)

html = f"""
<!DOCTYPE html>
<html>
<head>
<meta charset='utf-8'/>
<title>Per-well NI/I 488-channel viewer with multi-label localization</title>
<style>
body {{ font-family: Arial, sans-serif; margin: 10px; }}
#well-buttons button {{ margin: 2px; padding: 4px 8px; }}
.well-panel {{ display: none; margin-top: 10px; }}
.well-panel.active {{ display: block; }}
.cell-grid {{ display: flex; flex-wrap: wrap; gap: 6px; }}
.cell-card {{ border: 1px solid #ccc; padding: 4px; width: 120px; font-size: 11px; }}
.cell-card img {{ width: 100%; border: 1px solid #000; }}
.status-I img {{ border-color: red; }}
.status-NI img {{ border-color: #ffffff; }}
.label-select {{ width: 220px; margin-top: 2px; height: 80px; }}
#summary-btn {{ margin-top: 10px; }}
#summary-table {{ margin-top: 10px; border-collapse: collapse; width: 100%; font-size: 11px; }}
#summary-table th, #summary-table td {{ border: 1px solid #ccc; padding: 4px; text-align: center; }}
#circos-container {{ margin-top: 15px; }}
</style>
</head>
<body>
<h2>Per-well NI/I 488-channel viewer with localization labeling</h2>
<p>Each cell is shown as a 50x50 488-channel image (green). Borders: red = Infected (I), white = Non-infected (NI).<br>
Use the multi-select boxes to assign one or more localization labels for NI and I populations per well (Ctrl/Cmd-click to select multiple).</p>
<div id='well-buttons'></div>
<div id='well-panels'></div>
<button id='summary-btn'>Generate Summary & Circos-style Plot</button>
<div id='summary-area'></div>
<div id='circos-container'></div>

<script src='https://cdn.plot.ly/plotly-2.27.0.min.js'></script>
<script>
const sampledCells = {data_json};
const localizationLabels = {labels_json};

const wells = Array.from(new Set(sampledCells.map(c => c.well_id))).sort();

const wellButtonsDiv = document.getElementById('well-buttons');
const wellPanelsDiv = document.getElementById('well-panels');

wells.forEach((well, idx) => {{
  const btn = document.createElement('button');
  btn.textContent = well;
  btn.onclick = () => showWellPanel(well);
  wellButtonsDiv.appendChild(btn);

  const panel = document.createElement('div');
  panel.id = 'panel-' + well;
  panel.className = 'well-panel' + (idx === 0 ? ' active' : '');

  const title = document.createElement('h3');
  title.textContent = 'Well ' + well;
  panel.appendChild(title);

  const niLabel = document.createElement('div');
  niLabel.innerHTML = '<b>NI localization (multi-select):</b><br/>';
  const niSelect = document.createElement('select');
  niSelect.id = 'label-NI-' + well;
  niSelect.className = 'label-select';
  niSelect.multiple = true;
  localizationLabels.forEach(l => {{
    const opt = document.createElement('option');
    opt.value = l;
    opt.textContent = l;
    niSelect.appendChild(opt);
  }});
  niLabel.appendChild(niSelect);
  panel.appendChild(niLabel);

  const iLabel = document.createElement('div');
  iLabel.innerHTML = '<b>I localization (multi-select):</b><br/>';
  const iSelect = document.createElement('select');
  iSelect.id = 'label-I-' + well;
  iSelect.className = 'label-select';
  iSelect.multiple = true;
  localizationLabels.forEach(l => {{
    const opt = document.createElement('option');
    opt.value = l;
    opt.textContent = l;
    iSelect.appendChild(opt);
  }});
  iLabel.appendChild(iSelect);
  panel.appendChild(iLabel);

  const grid = document.createElement('div');
  grid.className = 'cell-grid';

  const cells = sampledCells.filter(c => c.well_id === well);
  cells.forEach(c => {{
    const card = document.createElement('div');
    card.className = 'cell-card status-' + c.infection_status;

    const img = document.createElement('img');
    img.src = 'data:image/png;base64,' + c.image;
    card.appendChild(img);

    const meta = document.createElement('div');
    meta.textContent = c.infection_status === 'I' ? 'Infected' : 'Non-infected';
    card.appendChild(meta);

    grid.appendChild(card);
  }});

  panel.appendChild(grid);
  wellPanelsDiv.appendChild(panel);
}});

function showWellPanel(well) {{
  document.querySelectorAll('.well-panel').forEach(p => p.classList.remove('active'));
  const panel = document.getElementById('panel-' + well);
  if (panel) panel.classList.add('active');
}}

function getSelectedOptions(selectEl) {{
  return Array.from(selectEl.options)
              .filter(o => o.selected)
              .map(o => o.value);
}}

function generateSummaryAndCircos() {{
  const summaryArea = document.getElementById('summary-area');
  const circosDiv = document.getElementById('circos-container');
  summaryArea.innerHTML = '';
  circosDiv.innerHTML = '';

  const rows = [];
  wells.forEach(well => {{
    const niSel = document.getElementById('label-NI-' + well);
    const iSel = document.getElementById('label-I-' + well);
    if (!niSel || !iSel) return;
    const niVals = getSelectedOptions(niSel);
    const iVals = getSelectedOptions(iSel);
    rows.push({{
      well: well,
      NI: niVals,
      I: iVals,
    }});
  }});

  let html = '<h3>Localization Summary</h3>';
  html += '<table id="summary-table">';
  html += '<tr><th>Well</th><th>NI localization</th><th>I localization</th></tr>';
  rows.forEach(r => {{
    html += `<tr><td>${{r.well}}</td><td>${{r.NI.join(', ')}}</td><td>${{r.I.join(', ')}}</td></tr>`;
  }});
  html += '</table>';
  summaryArea.innerHTML = html;

  const flowCounts = {{}};
  rows.forEach(r => {{
    r.NI.forEach(src => {{
      r.I.forEach(dst => {{
        const key = src + '->' + dst;
        if (!flowCounts[key]) flowCounts[key] = 0;
        flowCounts[key] += 1;
      }});
    }});
  }});

  const sources = [];
  const targets = [];
  const values = [];

  const nodes = [];
  const nodeIndex = {{}};

  localizationLabels.forEach((lab, i) => {{
    const name = lab + ' (NI)';
    nodeIndex['NI:' + lab] = nodes.length;
    nodes.push(name);
  }});
  localizationLabels.forEach((lab, i) => {{
    const name = lab + ' (I)';
    nodeIndex['I:' + lab] = nodes.length;
    nodes.push(name);
  }});

  Object.keys(flowCounts).forEach(k => {{
    const [src, dst] = k.split('->');
    const srcKey = 'NI:' + src;
    const dstKey = 'I:' + dst;
    if (!(srcKey in nodeIndex) || !(dstKey in nodeIndex)) return;
    sources.push(nodeIndex[srcKey]);
    targets.push(nodeIndex[dstKey]);
    values.push(flowCounts[k]);
  }});

  const palette = [
    '#1f77b4','#ff7f0e','#2ca02c','#d62728','#9467bd',
    '#8c564b','#e377c2','#7f7f7f','#bcbd22','#17becf',
    '#636efa','#EF553B'
  ];

  const nodeColors = nodes.map((_, idx) => palette[idx % palette.length]);

  const data = [{{
    type: 'sankey',
    orientation: 'h',
    node: {{
      pad: 15,
      thickness: 15,
      line: {{color: 'black', width: 0.5}},
      label: nodes,
      color: nodeColors,
    }},
    link: {{
      source: sources,
      target: targets,
      value: values,
      color: 'rgba(100,100,100,0.4)',
    }},
  }}];

  const layout = {{
    title: 'Protein subcellular localization: Non-infected (left) vs Infected (right)',
    font: {{size: 10}},
    height: 600,
  }};

  Plotly.newPlot(circosDiv, data, layout);
}}

document.getElementById('summary-btn').onclick = generateSummaryAndCircos;
</script>
</body>
</html>
"""

asyncio.ensure_future(api.create_window(src=html, name='NI_I_488_only_multilabel_and_circos_v2'))


Regenerating UI with correct 488-channel images and multi-select localization...
Sampled cells with images: 1804


<Future pending>

Here's my annotation:

--------
| Protein localization Non-infected cells (SCV = Single Cell Variation) | Protein localization Infected cells (SCV= Single Cell Variation) | Spatial/Intensity up/Intensity down | Well_id |
| Cytosol | Cytosol | - | 1_A1 |
| Nucleoplasm, Vesicles, Cytosol | Nucleoplasm, Vesicles, Cytosol | - | 1_A10 |
| Golgi Apparatus, Cytosol, SCV | Cytosol | Spatial | 1_A11 |
| Nucleoli, Endoplasmic reticulum | Nucleoli, Endoplasmic reticulum, Golgi Apparatus | Spatial, Intensity up | 1_A12 |
| Mitochondria, SCV | Mitochondria, SCV | Intensity down | 1_A2 |
| Cytosol, Endoplasmic reticulum | Cytosol, Endoplasmic reticulum, SCV | Intensity up | 1_A3 |
| Nucleoli, Vesicles, Plasma membrane, Cytosol, SCV | Nucleoli, Vesicles, Plasma membrane, Cytosol, SCV | - | 1_A4 |
| Vesicles | Vesicles | Intensity up | 1_A5 |
| Mitochondria, Nuclear speckles, Nucleoli fibrillar center | Mitochondria, Nuclear speckles, Nucleoli fibrillar center | Intensity down | 1_A6 |
| Vesicles | Vesicles | - | 1_A7 |
| Cytosol | Cytosol | - | 1_A8 |
| Nucleoplasm, Cytosol, SCV | Nucleoplasm, Cytosol, SCV | - | 1_B1 |
| Golgi Apparatus, Cytosol | Golgi Apparatus, Cytosol | - | 1_B10 |
| Microtubules | Microtubules, SCV | - | 1_B11 |
| Nuclear speckles, Cytosol, SCV | Nuclear speckles,  Endoplasmic reticulum,  Golgi Apparatus,  Cytosol,  SCV | Spatial, Intensity up | 1_B12 |
| Golgi Apparatus, Cytosol, SCV | Cytosol | Spatial | 1_B2 |
| Vesicles | Vesicles | - | 1_B3 |
| Cytosol | Endoplasmic reticulum, Golgi Apparatus, Vesicles, Cytosol, SCV | Spatial, Intensity up | 1_B4 |
| Nucleoplasm | Nucleoplasm | - | 1_B5 |
| Golgi Apparatus | Plasma membrane, Cytosol | Spatial, Intensity up | 1_B6 |
| Mitochondria | Mitochondria | - | 1_B7 |
| Cytosol | Cytosol, SCV | Intensity up | 1_B8 |
| Vesicles, Cytosol | Vesicles, Cytosol, SCV | - | 1_B9 |
| Vesicles | Vesicles | - | 1_C1 |
| Vesicles, SCV | Vesicles, Cytosol, SCV | Spatial, Intensity up | 1_C11 |
| Nucleoplasm, Vesicles, Cytosol, SCV | Nucleoplasm|Vesicles|Cytosol|SCV | - | 1_C12 |
| Vesicles | Vesicles | - | 1_C2 |
| Nucleoplasm, Golgi Apparatus, Vesicles | Nucleoplasm, Golgi Apparatus, Vesicles | - | 1_C3 |
| Endoplasmic reticulum, Cytosol | Endoplasmic reticulum, Cytosol | - | 1_C4 |
| Spindle, SCV | Spindle, SCV | - | 1_C5 |
| Vesicles, SCV | Vesicles, SCV | - | 1_C6 |
| Vesicles, SCV | Vesicles, SCV | - | 1_C7 |
| Nucleoplasm, Cytosol | Nucleoplasm, Endoplasmic reticulum, Golgi Apparatus, Cytosol, SCV | Spatial | 1_C8 |
| Vesicles, Cytosol | Vesicles, Cytosol, SCV | Intensity up | 1_D1 |
| Mitochondria, Cytosol | Mitochondria, Cytosol | - | 1_D10 |
| Nucleoplasm, Cytosol, SCV | Nucleoplasm, Cytosol, SCV | Intensity up | 1_D11 |
| Golgi Apparatus, Vesicles, Cytosol, SCV | Golgi Apparatus, Vesicles, Cytosol, Lipid drop, SCV | Spatial | 1_D12 |
| Negative | Negative | - | 1_D2 |
| Nucleoplasm, Vesicles, Plasma membrane, Aggresome, SCV | Nucleoplasm, Vesicles, Plasma membrane, Aggresome, SCV | - | 1_D3 |
| Nucleoplasm, Vesicles, SCV | Nucleoplasm, Vesicles, SCV | - | 1_D4 |
| Vesicles, Cytosol, SCV | Vesicles, Cytosol, SCV | - | 1_D5 |
| Nucleoplasm, Golgi Apparatus, Cytosol | Nucleoplasm, Golgi apparatus, Cytosol | - | 1_D6 |
| Nucleoplasm, Cytosol | Nucleoplasm, Cytosol | - | 1_D7 |
| Nucleoplasm, Golgi Apparatus, Vesicles, Plasma membrane, Cytosol, SCV | Nucleoplasm, Golgi Apparatus, Vesicles, Plasma membrane, Cytosol, SCV | - | 1_D8 |
| Vesicles | Vesicles | - | 1_D9 |
| Nucleoli, Mitochondria, Cytosol, SCV | Nucleoli, Mitochondria, Cytosol, SCV | - | 1_E1 |
| Nucleoplasm, Vesicles, Cytosol | Nucleoplasm, Vesicles, Cytosol | - | 1_E10 |
| Cytosol, SCV | Cytosol, SCV | - | 1_E11 |
| Cytosol, SCV | Cytosol, SCV | - | 1_E12 |
| Negative | Negative | - | 1_E2 |
| Endoplasmic reticulum, Nuclear membrane | Endoplasmic reticulum, Nuclear membrane | - | 1_E3 |
| Endoplasmic reticulum | Endoplasmic reticulum | - | 1_E4 |
| Golgi Apparatus | Golgi Apparatus | - | 1_E5 |
| Microtubules | Microtubules | - | 1_E7 |
| Nucleoplasm, Cytosol | Nucleoplasm, Cytosol | - | 1_E8 |
| Cytosol, SCV | Cytosol, SCV | - | 1_E9 |
| Nucleoplasm, SCV | Nucleoplasm, Endoplasmic reticulum, SCV | Spatial, Intensity up | 1_F1 |
| Vesicles | Vesicles | - | 1_F10 |
| Vesicles, Cytosol, SCV | Vesicles, Cytosol, SCV | - | 1_F12 |
| Negative | Negative | - | 1_F2 |
| Nucleoplasm, Golgi Apparatus, Vesicles | Nucleoplasm, Golgi Apparatus, Vesicles | - | 1_F4 |
| Nucleoplasm | Nucleoplasm | - | 1_F6 |
| Nucleoplasm, SCV | Nucleoplasm, SCV | - | 1_F7 |
| Nucleoplasm, Golgi Apparatus, Vesicles, Cytosol, SCV | Nucleoplasm, Golgi Apparatus, Vesicles, Cytosol, SCV | - | 1_F8 |
| Nucleoplasm, Golgi Apparatus, SCV | Nucleoplasm, Golgi Apparatus, SCV | Intensity up | 1_F9 |
| Cytosol | Cytosol | - | 1_G1 |
| Mitochondria | Mitochondria | - | 1_G10 |
| Endoplasmic reticulum, Cytosol | Endoplasmic reticulum, Cytosol, SCV | Intensity up | 1_G11 |
| Vesicles, SCV | Vesicles, SCV | - | 1_G12 |
| Cytosol | Cytosol | - | 1_G2 |
| Nucleoplasm, Nuclear membrane | Nucleoplasm, Nuclear membrane | - | 1_G4 |
| Nucleoplasm, Cytosol, SCV | Nucleoplasm, Cytosol, SCV | - | 1_G5 |
| Plasma membrane, SCV | Plasma membrane, SCV | - | 1_G6 |
| Cytosol | Cytosol | - | 1_G8 |
| Vesicles, Aggresome, SCV | Vesicles, Aggresome, SCV | - | 1_G9 |
| Vesicles, Cytosol | Vesicles, Cytosol | - | 1_H1 |
| Vesicles, Cytosol, SCV | Cytosol, SCV | Spatial, Intensity up | 1_H11 |
| Nucleoplasm, Nuclear speckles, Cytosol | Nucleoplasm, Nuclear speckles, Cytosol | - | 1_H3 |
| Negative | Negative | - | 1_H4 |
| Vesicles, SCV | Vesicles, SCV | - | 1_H6 |
| Nucleoplasm, Nuclear membrane, Vesicles | Nucleoplasm, Nuclear membrane, Vesicles | - | 1_H7 |
| Nucleoplasm, Cytosol, SCV | Nucleoplasm, Cytosol, SCV | - | 1_H8 |
| Vesicles, Cytosol, SCV | Vesicles, Cytosol, SCV | - | 1_H9 |
-----------

Please generate the same circos plot with the these annotation. Also SCV should not be displayed; it's not a subcellular organelle/location.


In [ ]:
import pandas as pd
import re

print('Parsing user-provided annotation table...')

raw_text = '''Protein localization Non-infected cells (SCV = Single Cell Variation)	Protein localization Infected cells (SCV= Single Cell Variation)	Spatial/Intensity up/Intensity down	Well_id
Cytosol	Cytosol	-	1_A1
Nucleoplasm, Vesicles, Cytosol	Nucleoplasm, Vesicles, Cytosol	-	1_A10
Golgi Apparatus, Cytosol, SCV	Cytosol	Spatial	1_A11
Nucleoli, Endoplasmic reticulum	Nucleoli, Endoplasmic reticulum, Golgi Apparatus	Spatial, Intensity up	1_A12
Mitochondria, SCV	Mitochondria, SCV	Intensity down	1_A2
Cytosol, Endoplasmic reticulum	Cytosol, Endoplasmic reticulum, SCV	Intensity up	1_A3
Nucleoli, Vesicles, Plasma membrane, Cytosol, SCV	Nucleoli, Vesicles, Plasma membrane, Cytosol, SCV	-	1_A4
Vesicles	Vesicles	Intensity up	1_A5
Mitochondria, Nuclear speckles, Nucleoli fibrillar center	Mitochondria, Nuclear speckles, Nucleoli fibrillar center	Intensity down	1_A6
Vesicles	Vesicles	-	1_A7
Cytosol	Cytosol	-	1_A8
Nucleoplasm, Cytosol, SCV	Nucleoplasm, Cytosol, SCV	-	1_B1
Golgi Apparatus, Cytosol	Golgi Apparatus, Cytosol	-	1_B10
Microtubules	Microtubules, SCV	-	1_B11
Nuclear speckles, Cytosol, SCV	Nuclear speckles,  Endoplasmic reticulum,  Golgi Apparatus,  Cytosol,  SCV	Spatial, Intensity up	1_B12
Golgi Apparatus, Cytosol, SCV	Cytosol	Spatial	1_B2
Vesicles	Vesicles	-	1_B3
Cytosol	Endoplasmic reticulum, Golgi Apparatus, Vesicles, Cytosol, SCV	Spatial, Intensity up	1_B4
Nucleoplasm	Nucleoplasm	-	1_B5
Golgi Apparatus	Plasma membrane, Cytosol	Spatial, Intensity up	1_B6
Mitochondria	Mitochondria	-	1_B7
Cytosol	Cytosol, SCV	Intensity up	1_B8
Vesicles, Cytosol	Vesicles, Cytosol, SCV	-	1_B9
Vesicles	Vesicles	-	1_C1
Vesicles, SCV	Vesicles, Cytosol, SCV	Spatial, Intensity up	1_C11
Nucleoplasm, Vesicles, Cytosol, SCV	Nucleoplasm|Vesicles|Cytosol|SCV	-	1_C12
Vesicles	Vesicles	-	1_C2
Nucleoplasm, Golgi Apparatus, Vesicles	Nucleoplasm, Golgi Apparatus, Vesicles	-	1_C3
Endoplasmic reticulum, Cytosol	Endoplasmic reticulum, Cytosol	-	1_C4
Spindle, SCV	Spindle, SCV	-	1_C5
Vesicles, SCV	Vesicles, SCV	-	1_C6
Vesicles, SCV	Vesicles, SCV	-	1_C7
Nucleoplasm, Cytosol	Nucleoplasm, Endoplasmic reticulum, Golgi Apparatus, Cytosol, SCV	Spatial	1_C8
Vesicles, Cytosol	Vesicles, Cytosol, SCV	Intensity up	1_D1
Mitochondria, Cytosol	Mitochondria, Cytosol	-	1_D10
Nucleoplasm, Cytosol, SCV	Nucleoplasm, Cytosol, SCV	Intensity up	1_D11
Golgi Apparatus, Vesicles, Cytosol, SCV	Golgi Apparatus, Vesicles, Cytosol, Lipid drop, SCV	Spatial	1_D12
Negative	Negative	-	1_D2
Nucleoplasm, Vesicles, Plasma membrane, Aggresome, SCV	Nucleoplasm, Vesicles, Plasma membrane, Aggresome, SCV	-	1_D3
Nucleoplasm, Vesicles, SCV	Nucleoplasm, Vesicles, SCV	-	1_D4
Vesicles, Cytosol, SCV	Vesicles, Cytosol, SCV	-	1_D5
Nucleoplasm, Golgi Apparatus, Cytosol	Nucleoplasm, Golgi apparatus, Cytosol	-	1_D6
Nucleoplasm, Cytosol	Nucleoplasm, Cytosol	-	1_D7
Nucleoplasm, Golgi Apparatus, Vesicles, Plasma membrane, Cytosol, SCV	Nucleoplasm, Golgi Apparatus, Vesicles, Plasma membrane, Cytosol, SCV	-	1_D8
Vesicles	Vesicles	-	1_D9
Nucleoli, Mitochondria, Cytosol, SCV	Nucleoli, Mitochondria, Cytosol, SCV	-	1_E1
Nucleoplasm, Vesicles, Cytosol	Nucleoplasm, Vesicles, Cytosol	-	1_E10
Cytosol, SCV	Cytosol, SCV	-	1_E11
Cytosol, SCV	Cytosol, SCV	-	1_E12
Negative	Negative	-	1_E2
Endoplasmic reticulum, Nuclear membrane	Endoplasmic reticulum, Nuclear membrane	-	1_E3
Endoplasmic reticulum	Endoplasmic reticulum	-	1_E4
Golgi Apparatus	Golgi Apparatus	-	1_E5
Microtubules	Microtubules	-	1_E7
Nucleoplasm, Cytosol	Nucleoplasm, Cytosol	-	1_E8
Cytosol, SCV	Cytosol, SCV	-	1_E9
Nucleoplasm, SCV	Nucleoplasm, Endoplasmic reticulum, SCV	Spatial, Intensity up	1_F1
Vesicles	Vesicles	-	1_F10
Vesicles, Cytosol, SCV	Vesicles, Cytosol, SCV	-	1_F12
Negative	Negative	-	1_F2
Nucleoplasm, Golgi Apparatus, Vesicles	Nucleoplasm, Golgi Apparatus, Vesicles	-	1_F4
Nucleoplasm	Nucleoplasm	-	1_F6
Nucleoplasm, SCV	Nucleoplasm, SCV	-	1_F7
Nucleoplasm, Golgi Apparatus, Vesicles, Cytosol, SCV	Nucleoplasm, Golgi Apparatus, Vesicles, Cytosol, SCV	-	1_F8
Nucleoplasm, Golgi Apparatus, SCV	Nucleoplasm, Golgi Apparatus, SCV	Intensity up	1_F9
Cytosol	Cytosol	-	1_G1
Mitochondria	Mitochondria	-	1_G10
Endoplasmic reticulum, Cytosol	Endoplasmic reticulum, Cytosol, SCV	Intensity up	1_G11
Vesicles, SCV	Vesicles, SCV	-	1_G12
Cytosol	Cytosol	-	1_G2
Nucleoplasm, Nuclear membrane	Nucleoplasm, Nuclear membrane	-	1_G4
Nucleoplasm, Cytosol, SCV	Nucleoplasm, Cytosol, SCV	-	1_G5
Plasma membrane, SCV	Plasma membrane, SCV	-	1_G6
Cytosol	Cytosol	-	1_G8
Vesicles, Aggresome, SCV	Vesicles, Aggresome, SCV	-	1_G9
Vesicles, Cytosol	Vesicles, Cytosol	-	1_H1
Vesicles, Cytosol, SCV	Cytosol, SCV	Spatial, Intensity up	1_H11
Nucleoplasm, Nuclear speckles, Cytosol	Nucleoplasm, Nuclear speckles, Cytosol	-	1_H3
Negative	Negative	-	1_H4
Vesicles, SCV	Vesicles, SCV	-	1_H6
Nucleoplasm, Nuclear membrane, Vesicles	Nucleoplasm, Nuclear membrane, Vesicles	-	1_H7
Nucleoplasm, Cytosol, SCV	Nucleoplasm, Cytosol, SCV	-	1_H8
Vesicles, Cytosol, SCV	Vesicles, Cytosol, SCV	-	1_H9
'''

from io import StringIO

df_annot = pd.read_csv(StringIO(raw_text), sep='\t')
print('Annotation rows:', len(df_annot))

for col in df_annot.columns[:2]:
    df_annot[col] = df_annot[col].astype(str).str.replace('|', ', ', regex=False)

loc_cols = [df_annot.columns[0], df_annot.columns[1]]

cleaned = []
for _, row in df_annot.iterrows():
    ni_raw = row[loc_cols[0]]
    i_raw = row[loc_cols[1]]
    def split_and_clean(s):
        parts = re.split(r',', str(s))
        parts = [p.strip() for p in parts if p.strip()]
        parts = [p for p in parts if p.upper() != 'SCV']
        return parts
    ni_list = split_and_clean(ni_raw)
    i_list = split_and_clean(i_raw)
    cleaned.append({
        'well': row['Well_id'],
        'NI_list': ni_list,
        'I_list': i_list,
        'change': row[df_annot.columns[2]],
    })

annot_clean = pd.DataFrame(cleaned)
print('Cleaned annotation rows:', len(annot_clean))
print('Example row:', annot_clean.iloc[0].to_dict())

from collections import Counter

flow_counts = Counter()
for _, r in annot_clean.iterrows():
    for src in r['NI_list']:
        for dst in r['I_list']:
            flow_counts[(src, dst)] += 1

print('Number of unique NI->I flows:', len(flow_counts))

all_locs = sorted(set([s for (s, _) in flow_counts.keys()] + [d for (_, d) in flow_counts.keys()]))
print('Unique localization labels (without SCV):', all_locs)

import plotly.graph_objects as go

nodes = []
node_index = {}
for lab in all_locs:
    key = f'NI:{lab}'
    node_index[key] = len(nodes)
    nodes.append(f'{lab} (NI)')
for lab in all_locs:
    key = f'I:{lab}'
    node_index[key] = len(nodes)
    nodes.append(f'{lab} (I)')

sources = []
targets = []
values = []
for (src, dst), v in flow_counts.items():
    s_key = f'NI:{src}'
    t_key = f'I:{dst}'
    if s_key in node_index and t_key in node_index and v > 0:
        sources.append(node_index[s_key])
        targets.append(node_index[t_key])
        values.append(v)

palette = [
    '#1f77b4','#ff7f0e','#2ca02c','#d62728','#9467bd',
    '#8c564b','#e377c2','#7f7f7f','#bcbd22','#17becf',
    '#636efa','#EF553B','#00cc96','#ab63fa','#FFA15A',
]
node_colors = [palette[i % len(palette)] for i in range(len(nodes))]

fig = go.Figure(data=[go.Sankey(
    arrangement='snap',
    node=dict(
        pad=15,
        thickness=15,
        line=dict(color='black', width=0.5),
        label=nodes,
        color=node_colors,
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color='rgba(120,120,120,0.4)',
    )
)])

fig.update_layout(
    title_text='Protein subcellular localization transitions: Non-infected (left) → Infected (right)',
    font_size=10,
    height=700,
)

from io import StringIO as _StringIO
import base64, textwrap

html_buf = _StringIO()
fig.write_html(html_buf, include_plotlyjs='cdn', full_html=True)
html_str = html_buf.getvalue()

asyncio.ensure_future(api.create_window(src=html_str, name='User_Annotated_Localization_Sankey'))

print('Summary table and Sankey-style Circos plot generated.')

Parsing user-provided annotation table...
Annotation rows: 83
Cleaned annotation rows: 83
Example row: {'well': '1_A1', 'NI_list': ['Cytosol'], 'I_list': ['Cytosol'], 'change': '-'}
Number of unique NI->I flows: 82
Unique localization labels (without SCV): ['Aggresome', 'Cytosol', 'Endoplasmic reticulum', 'Golgi Apparatus', 'Golgi apparatus', 'Lipid drop', 'Microtubules', 'Mitochondria', 'Negative', 'Nuclear membrane', 'Nuclear speckles', 'Nucleoli', 'Nucleoli fibrillar center', 'Nucleoplasm', 'Plasma membrane', 'Spindle', 'Vesicles']
Summary table and Sankey-style Circos plot generated.


Use the same style of circos plot. also SCV should not be displayed, it's not a subcellular organelle/location.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from io import BytesIO
import base64

print('Rebuilding circos-style plot without SCV labels...')

raw_table = '''Protein localization Non-infected cells (SCV = Single Cell Variation)	Protein localization Infected cells (SCV= Single Cell Variation)	Spatial/Intensity up/Intensity down	Well_id
Cytosol	Cytosol	-	1_A1
Nucleoplasm, Vesicles, Cytosol	Nucleoplasm, Vesicles, Cytosol	-	1_A10
Golgi Apparatus, Cytosol, SCV	Cytosol	Spatial	1_A11
Nucleoli, Endoplasmic reticulum	Nucleoli, Endoplasmic reticulum, Golgi Apparatus	Spatial, Intensity up	1_A12
Mitochondria, SCV	Mitochondria, SCV	Intensity down	1_A2
Cytosol, Endoplasmic reticulum	Cytosol, Endoplasmic reticulum, SCV	Intensity up	1_A3
Nucleoli, Vesicles, Plasma membrane, Cytosol, SCV	Nucleoli, Vesicles, Plasma membrane, Cytosol, SCV	-	1_A4
Vesicles	Vesicles	Intensity up	1_A5
Mitochondria, Nuclear speckles, Nucleoli fibrillar center	Mitochondria, Nuclear speckles, Nucleoli fibrillar center	Intensity down	1_A6
Vesicles	Vesicles	-	1_A7
Cytosol	Cytosol	-	1_A8
Nucleoplasm, Cytosol, SCV	Nucleoplasm, Cytosol, SCV	-	1_B1
Golgi Apparatus, Cytosol	Golgi Apparatus, Cytosol	-	1_B10
Microtubules	Microtubules, SCV	-	1_B11
Nuclear speckles, Cytosol, SCV	Nuclear speckles,  Endoplasmic reticulum,  Golgi Apparatus,  Cytosol,  SCV	Spatial, Intensity up	1_B12
Golgi Apparatus, Cytosol, SCV	Cytosol	Spatial	1_B2
Vesicles	Vesicles	-	1_B3
Cytosol	Endoplasmic reticulum, Golgi Apparatus, Vesicles, Cytosol, SCV	Spatial, Intensity up	1_B4
Nucleoplasm	Nucleoplasm	-	1_B5
Golgi Apparatus	Plasma membrane, Cytosol	Spatial, Intensity up	1_B6
Mitochondria	Mitochondria	-	1_B7
Cytosol	Cytosol, SCV	Intensity up	1_B8
Vesicles, Cytosol	Vesicles, Cytosol, SCV	-	1_B9
Vesicles	Vesicles	-	1_C1
Vesicles, SCV	Vesicles, Cytosol, SCV	Spatial, Intensity up	1_C11
Nucleoplasm, Vesicles, Cytosol, SCV	Nucleoplasm|Vesicles|Cytosol|SCV	-	1_C12
Vesicles	Vesicles	-	1_C2
Nucleoplasm, Golgi Apparatus, Vesicles	Nucleoplasm, Golgi Apparatus, Vesicles	-	1_C3
Endoplasmic reticulum, Cytosol	Endoplasmic reticulum, Cytosol	-	1_C4
Spindle, SCV	Spindle, SCV	-	1_C5
Vesicles, SCV	Vesicles, SCV	-	1_C6
Vesicles, SCV	Vesicles, SCV	-	1_C7
Nucleoplasm, Cytosol	Nucleoplasm, Endoplasmic reticulum, Golgi Apparatus, Cytosol, SCV	Spatial	1_C8
Vesicles, Cytosol	Vesicles, Cytosol, SCV	Intensity up	1_D1
Mitochondria, Cytosol	Mitochondria, Cytosol	-	1_D10
Nucleoplasm, Cytosol, SCV	Nucleoplasm, Cytosol, SCV	Intensity up	1_D11
Golgi Apparatus, Vesicles, Cytosol, SCV	Golgi Apparatus, Vesicles, Cytosol, Lipid drop, SCV	Spatial	1_D12
Negative	Negative	-	1_D2
Nucleoplasm, Vesicles, Plasma membrane, Aggresome, SCV	Nucleoplasm, Vesicles, Plasma membrane, Aggresome, SCV	-	1_D3
Nucleoplasm, Vesicles, SCV	Nucleoplasm, Vesicles, SCV	-	1_D4
Vesicles, Cytosol, SCV	Vesicles, Cytosol, SCV	-	1_D5
Nucleoplasm, Golgi Apparatus, Cytosol	Nucleoplasm, Golgi apparatus, Cytosol	-	1_D6
Nucleoplasm, Cytosol	Nucleoplasm, Cytosol	-	1_D7
Nucleoplasm, Golgi Apparatus, Vesicles, Plasma membrane, Cytosol, SCV	Nucleoplasm, Golgi Apparatus, Vesicles, Plasma membrane, Cytosol, SCV	-	1_D8
Vesicles	Vesicles	-	1_D9
Nucleoli, Mitochondria, Cytosol, SCV	Nucleoli, Mitochondria, Cytosol, SCV	-	1_E1
Nucleoplasm, Vesicles, Cytosol	Nucleoplasm, Vesicles, Cytosol	-	1_E10
Cytosol, SCV	Cytosol, SCV	-	1_E11
Cytosol, SCV	Cytosol, SCV	-	1_E12
Negative	Negative	-	1_E2
Endoplasmic reticulum, Nuclear membrane	Endoplasmic reticulum, Nuclear membrane	-	1_E3
Endoplasmic reticulum	Endoplasmic reticulum	-	1_E4
Golgi Apparatus	Golgi Apparatus	-	1_E5
Microtubules	Microtubules	-	1_E7
Nucleoplasm, Cytosol	Nucleoplasm, Cytosol	-	1_E8
Cytosol, SCV	Cytosol, SCV	-	1_E9
Nucleoplasm, SCV	Nucleoplasm, Endoplasmic reticulum, SCV	Spatial, Intensity up	1_F1
Vesicles	Vesicles	-	1_F10
Vesicles, Cytosol, SCV	Vesicles, Cytosol, SCV	-	1_F12
Negative	Negative	-	1_F2
Nucleoplasm, Golgi Apparatus, Vesicles	Nucleoplasm, Golgi Apparatus, Vesicles	-	1_F4
Nucleoplasm	Nucleoplasm	-	1_F6
Nucleoplasm, SCV	Nucleoplasm, SCV	-	1_F7
Nucleoplasm, Golgi Apparatus, Vesicles, Cytosol, SCV	Nucleoplasm, Golgi Apparatus, Vesicles, Cytosol, SCV	-	1_F8
Nucleoplasm, Golgi Apparatus, SCV	Nucleoplasm, Golgi Apparatus, SCV	Intensity up	1_F9
Cytosol	Cytosol	-	1_G1
Mitochondria	Mitochondria	-	1_G10
Endoplasmic reticulum, Cytosol	Endoplasmic reticulum, Cytosol, SCV	Intensity up	1_G11
Vesicles, SCV	Vesicles, SCV	-	1_G12
Cytosol	Cytosol	-	1_G2
Nucleoplasm, Nuclear membrane	Nucleoplasm, Nuclear membrane	-	1_G4
Nucleoplasm, Cytosol, SCV	Nucleoplasm, Cytosol, SCV	-	1_G5
Plasma membrane, SCV	Plasma membrane, SCV	-	1_G6
Cytosol	Cytosol	-	1_G8
Vesicles, Aggresome, SCV	Vesicles, Aggresome, SCV	-	1_G9
Vesicles, Cytosol	Vesicles, Cytosol	-	1_H1
Vesicles, Cytosol, SCV	Cytosol, SCV	Spatial, Intensity up	1_H11
Nucleoplasm, Nuclear speckles, Cytosol	Nucleoplasm, Nuclear speckles, Cytosol	-	1_H3
Negative	Negative	-	1_H4
Vesicles, SCV	Vesicles, SCV	-	1_H6
Nucleoplasm, Nuclear membrane, Vesicles	Nucleoplasm, Nuclear membrane, Vesicles	-	1_H7
Nucleoplasm, Cytosol, SCV	Nucleoplasm, Cytosol, SCV	-	1_H8
Vesicles, Cytosol, SCV	Vesicles, Cytosol, SCV	-	1_H9
'''

rows = [r for r in raw_table.strip().split('\n')]
header = rows[0].split('\t')
records = [r.split('\t') for r in rows[1:]]
df = pd.DataFrame(records, columns=header)

ni_col = header[0]
i_col = header[1]

replace_map = {
    'Nucleoli fibrillar center': 'Nucleoli',
    'Lipid drop': 'Vesicles',
    'Golgi apparatus': 'Golgi Apparatus',
}

for col in [ni_col, i_col]:
    df[col] = df[col].str.replace('|', ',', regex=False)
    for bad, good in replace_map.items():
        df[col] = df[col].str.replace(bad, good, regex=False)


ignore_token = 'SCV'

def split_locs_filtered(s):
    parts = [p.strip() for p in s.split(',') if p.strip()]
    parts = [p for p in parts if ignore_token not in p]
    return parts


all_pairs = {}
for _, row in df.iterrows():
    ni_locs = split_locs_filtered(row[ni_col])
    i_locs = split_locs_filtered(row[i_col])
    if not ni_locs or not i_locs:
        continue
    for src in ni_locs:
        for dst in i_locs:
            key = (src, dst)
            all_pairs[key] = all_pairs.get(key, 0) + 1

if not all_pairs:
    print('No localization pairs after filtering SCV; nothing to plot.')
else:
    nodes = set()
    for (src, dst) in all_pairs.keys():
        nodes.add(('NI', src))
        nodes.add(('I', dst))

    nodes = sorted(nodes, key=lambda x: (x[0], x[1]))
    node_to_idx = {n: i for i, n in enumerate(nodes)}
    N = len(nodes)

    flows = []
    for (src, dst), w in all_pairs.items():
        flows.append((('NI', src), ('I', dst), w))
    flows = sorted(flows, key=lambda x: -x[2])

    base_radius = 1.0
    bar_thickness = 0.1
    link_alpha = 0.25

    palette = [
        '#1f77b4','#ff7f0e','#2ca02c','#d62728','#9467bd',
        '#8c564b','#e377c2','#7f7f7f','#bcbd22','#17becf',
        '#636efa','#EF553B'
    ]

    ang = np.linspace(0, 2*np.pi, N, endpoint=False)

    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': 'polar'})

    label_to_color = {}
    for i, (side, loc) in enumerate(nodes):
        if loc not in label_to_color:
            label_to_color[loc] = palette[len(label_to_color) % len(palette)]

    for i, (side, loc) in enumerate(nodes):
        theta = ang[i]
        r0 = base_radius
        r1 = base_radius + bar_thickness
        color = label_to_color[loc]
        ax.bar(theta, bar_thickness, width=2*np.pi/N*0.9, bottom=r0, color=color, edgecolor='black', linewidth=0.5)

    max_w = max(all_pairs.values())
    for (src_node, dst_node, w) in flows:
        i = node_to_idx[src_node]
        j = node_to_idx[dst_node]
        t1 = ang[i]
        t2 = ang[j]
        num = max(3, int(6 * w / max_w))
        ts = np.linspace(t1, t2, num)
        rs = np.linspace(base_radius + bar_thickness, base_radius + 0.3, num)
        ax.plot(ts, rs, color='gray', alpha=link_alpha * (0.5 + 0.5 * w / max_w), linewidth=0.5 + 1.5 * w / max_w)

    for i, (side, loc) in enumerate(nodes):
        theta = ang[i]
        r = base_radius + bar_thickness + 0.05
        rotation = np.degrees(theta)
        if np.pi/2 < theta < 3*np.pi/2:
            rotation = rotation + 180
            alignment = 'right'
        else:
            alignment = 'left'
        text = f"{loc} ({side})"
        ax.text(theta, r, text, rotation=rotation, rotation_mode='anchor',
                ha=alignment, va='center', fontsize=7)

    ax.set_rticks([])
    ax.set_xticks([])
    ax.set_yticklabels([])
    ax.set_title('Protein subcellular localization transitions: Non-infected (NI, left) to Infected (I, right)\n(SCV removed)', fontsize=10)

    buf = BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight', dpi=200)
    plt.close(fig)
    img_b64 = base64.b64encode(buf.getvalue()).decode('utf-8')
    html = f"<html><body><img src='data:image/png;base64,{img_b64}' style='max-width:100%;height:auto;'/></body></html>"
    asyncio.ensure_future(api.create_window(src=html, name='circos_style_localization_NI_to_I_noSCV'))
    print('Updated circos-style plot generated (SCV filtered out).')


Rebuilding circos-style plot without SCV labels...
Updated circos-style plot generated (SCV filtered out).
